# Ananylse de la population des intellectuels de la classe yago/Intellectual109621545 (DBpedia)

## Importation des librairies

In [2]:
from SPARQLWrapper import SPARQLWrapper, SPARQLWrapper2, JSON, TURTLE, XML, RDFXML
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import pprint
import csv
import ast
import re

from collections import Counter
from operator import itemgetter

import sqlite3 as sql
import time

from importlib import reload
from shutil import copyfile

In [4]:
import sparql_functions as spqf

In [5]:
reload(spqf)

<module 'sparql_functions' from 'C:\\jupyter_notebooks\\JS_intellectuels\\sparql_functions.py'>

## Création de la base de données 

In [5]:
cn = sql.connect('data/intellectuels.db')

In [116]:
c = cn.cursor()
c.execute("SELECT datetime('now', 'localtime')")
print(c.fetchone())

('2021-06-17 00:02:17',)


In [117]:
cn.close()

In [6]:
db = 'data/intellectuels.db'

In [120]:
cn = sql.connect(db)
c = cn.cursor()
# Create table QUERY
# c.execute('''DROP TABLE query;''')

c.execute('''CREATE TABLE query (pk_query INTEGER PRIMARY KEY,
label VARCHAR (100),
description TEXT,
project VARCHAR (150),
sparql_endpoint VARCHAR (250),
query TEXT,
notes TEXT, 
timestmp text DEFAULT ((datetime('now','localtime'))))
''')

# Save (commit) the changes
# désactivé # cn.commit()
cn.close()

In [121]:
cn = sql.connect(db)
c = cn.cursor()

# Create table RESULT
# c.execute('''DROP TABLE result;''')

c.execute('''CREATE TABLE result (pk_result INTEGER PRIMARY KEY,
fk_query INTEGER REFERENCES "query" (pk_query) MATCH SIMPLE,
description TEXT,
result TEXT,
insert_data_into TEXT,
notes TEXT, 
timestmp text DEFAULT ((datetime('now','localtime'))))
''')

# Save (commit) the changes
# désactivé # cn.commit()
cn.close()

In [122]:
cn = sql.connect(db)
c = cn.cursor()

# Create table 'entity'
# STOP #  c.execute('''DROP TABLE entity''')

c.execute('''CREATE TABLE entity
             (pk_entity INTEGER PRIMARY KEY, local_authority BOOLEAN DEFAULT TRUE, 
             fk_same_entity INTEGER REFERENCES [entity] (pk_entity) MATCH SIMPLE,
             uri_entity TEXT, entity_class TEXT, source TEXT,
             fk_query_as_source INTEGER REFERENCES [query] (pk_query) MATCH SIMPLE,
             entity_std_name TEXT, notes TEXT, 
             timestmp text DEFAULT ((datetime('now','localtime'))),
             UNIQUE (uri_entity, entity_class, source))''')
#"""

# Save (commit) the changes 
# STOP #   cn.commit()
cn.close()

In [123]:
cn = sql.connect(db)
c = cn.cursor()

# Create table 'property'
# STOP # c.execute('''DROP TABLE property''')

c.execute('''CREATE TABLE property
             (pk_property INTEGER PRIMARY KEY, 
             fk_entity INTEGER REFERENCES [entity] (pk_entity) MATCH SIMPLE,
             uri_entity TEXT, property TEXT, value TEXT, 
             source TEXT, quality INTEGER, fk_query_as_source INTEGER REFERENCES [query] (pk_query) MATCH SIMPLE,
             additional_property_1 TEXT, value_1 TEXT,
             additional_property_2 TEXT, value_2 TEXT,
             additional_property_3 TEXT, value_3 TEXT,
             notes TEXT, 
             timestmp text DEFAULT ((datetime('now','localtime'))),
             UNIQUE (uri_entity, property, value, source))''')
#"""

# Save (commit) the changes 
# STOP # cn.commit()
cn.close()

## Récupération des données

In [91]:
query_count_intellectuals = """
PREFIX  yago: <http://dbpedia.org/class/yago/>

SELECT DISTINCT (count(*) AS ?eff)
WHERE
  {   
       SELECT DISTINCT ?person 
{ ?person  a                     yago:Intellectual109621545 }
}

"""

In [93]:
query_intellectuals_properties = """
PREFIX  yago: <http://dbpedia.org/class/yago/>
PREFIX  foaf: <http://xmlns.com/foaf/0.1/>

SELECT DISTINCT ?p ?direction (count(*) AS ?freq)
WHERE
  {  { ?person  a                     yago:Intellectual109621545 ;
                     ?p                    ?o
            BIND("out" AS ?direction)
          }
     
    UNION
          { ?person  a                     yago:Intellectual109621545 .
            ?s       ?p                    ?person
            BIND("in" AS ?direction)
          }      
  }
GROUP BY ?p ?direction
ORDER BY DESC(?freq)

"""

### Compter les intellectuels

In [94]:
# Exécution de la requête qui retournera un dictionnaire JSON
sparql = SPARQLWrapper("https://dbpedia.org/sparql")
sparql.setQuery(query_count_intellectuals)
sparql.setReturnFormat(JSON)
rc = sparql.queryAndConvert()
rc

{'head': {'link': [], 'vars': ['eff']},
 'results': {'distinct': False,
  'ordered': True,
  'bindings': [{'eff': {'type': 'typed-literal',
     'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
     'value': '197176'}}]}}

## Enregistrer les entitées 'person'

In [124]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 2

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(2,
 'Cherches les intellectuels ',
 'Cherches les intellectuels (1-10000)',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?person     \r\nWHERE\r\n  \r\n{ ?person  a                     yago:Intellectual109621545 }\r\nORDER BY ?person',
 None,
 '2021-06-17 00:10:56')

In [125]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [126]:
len(qr['results']['bindings'])

10000

In [127]:
r = [l for l in spqf.sparql_result_to_list(qr) ]

In [128]:
rl = [[l[0], 'https://dbpedia.org/ontology/Person', 'https://dbpedia.org/sparql', 2 ] for l in r]

In [129]:
rl[:3]

[["http://dbpedia.org/resource/'Abd_al-'Aziz_al-Wafa'i",
  'https://dbpedia.org/ontology/Person',
  'https://dbpedia.org/sparql',
  2],
 ["http://dbpedia.org/resource/'Abd_al-Razzaq_al-Hasani",
  'https://dbpedia.org/ontology/Person',
  'https://dbpedia.org/sparql',
  2],
 ["http://dbpedia.org/resource/'Ali_ibn_al-'Abbas_al-Majusi",
  'https://dbpedia.org/ontology/Person',
  'https://dbpedia.org/sparql',
  2]]

In [130]:
### Stocker les nouvelles entités en base de données – ne pas oublier d'active le 'commit' !

cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO entity (uri_entity, entity_class, source, fk_query_as_source) VALUES (?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! ##  cn.commit()

cn.close()

### Sauvegarde DB

Copie1 = entités person enregistrées

In [269]:
### Fonction qui récupère et met en forme le temps au moment de l'exécution

# définition
def timestamp_formatted_for_file_name():
    is_now = time.strftime('%Y%m%d_%H%M%S')
    return is_now

# exécution
timestamp_formatted_for_file_name()

'20210617_011212'

In [270]:
### Définir les adresses des fichiers, l'existant et celui à créer
original_db = 'data/intellectuels.db'

##  Noter que la différence de suffixe, en soi totalement arbitraire, 
#  dépend du fait que dans la configuration de .gitignore, .sqlite est exclu du verisonnement GIT
#  contrairement à .db qui est versionné
timestamped_db_copy = 'data/intellectuels_copie1_' + timestamp_formatted_for_file_name() + '.sqlite'

In [271]:
### Créer une copie de sauvegarde avec timestamp du fichier (versionnement manuel)
# ATTENTION : la base de données doit exister à l'endroit indiqué !
## Cette requête n'est utile que si des modifications en écriture vont être apportées à la base de données,
# afin de préserver le dernier état avant modification

## Documentation:
# https://docs.python.org/3/library/shutil.htmlcopied_db = copyfile(original_db, timestamped_db_copy)

copied_db = copyfile(original_db, timestamped_db_copy)
copied_db

'data/intellectuels_copie1_20210617_011241.sqlite'

### Inspection

In [272]:
# connnexion à la base de données
cn = sql.connect(db)
c = cn.cursor()

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c.execute('SELECT count(*) AS eff, entity_class FROM entity GROUP BY entity_class')
result_q = c.fetchall()

# fermer la connexion
cn.close()
result_q[:3]

[(197176, 'https://dbpedia.org/ontology/Person')]

Les 197'176 intellectuels sont enregitrés dans la base de données !

### Sortir la liste des propriétés des intellectuels

In [44]:
# Exécution de la requête qui retournera un dictionnaire JSON
sparql = SPARQLWrapper("https://dbpedia.org/sparql")
sparql.setQuery(query_intellectuals_properties)
sparql.setReturnFormat(JSON)
rc = sparql.queryAndConvert()


# Transformation des résultats en liste puis en dataframe comprenant les propriétés et leurs fréquences
l_prop = spqf.sparql_result_to_list_vars(rc, '?p ?freq')
df_prop = pd.DataFrame(l_prop, columns=['Property','Frequency'])

# Affichage des 50 propriétés les mieux renseignées
df_prop[:50]

,Property,Frequency
0,http://dbpedia.org/ontology/wikiPageWikiLink,9405330
1,http://dbpedia.org/ontology/wikiPageWikiLink,8047534
2,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,6697687
3,http://www.w3.org/2002/07/owl#sameAs,2618194
4,http://purl.org/dc/terms/subject,2129616
5,http://dbpedia.org/property/wikiPageUsesTemplate,1658868
6,http://www.w3.org/2000/01/rdf-schema#label,862286
7,http://www.w3.org/2000/01/rdf-schema#comment,850919
8,http://dbpedia.org/ontology/abstract,850919
9,http://dbpedia.org/ontology/wikiPageExternalLink,508640


## Propriétés

### Sauvegarde DB

Copie2 = entités person enregistrées / requêtes birthyear, birthdate ajoutées

In [5]:
### Fonction qui récupère et met en forme le temps au moment de l'exécution

# définition
def timestamp_formatted_for_file_name():
    is_now = time.strftime('%Y%m%d_%H%M%S')
    return is_now

# exécution
timestamp_formatted_for_file_name()

'20210704_194539'

In [6]:
### Définir les adresses des fichiers, l'existant et celui à créer
original_db = 'data/intellectuels.db'

##  Noter que la différence de suffixe, en soi totalement arbitraire, 
#  dépend du fait que dans la configuration de .gitignore, .sqlite est exclu du verisonnement GIT
#  contrairement à .db qui est versionné
timestamped_db_copy = 'data/intellectuels_copie2_' + timestamp_formatted_for_file_name() + '.sqlite'

In [7]:
### Créer une copie de sauvegarde avec timestamp du fichier (versionnement manuel)
# ATTENTION : la base de données doit exister à l'endroit indiqué !
## Cette requête n'est utile que si des modifications en écriture vont être apportées à la base de données,
# afin de préserver le dernier état avant modification

## Documentation:
# https://docs.python.org/3/library/shutil.htmlcopied_db = copyfile(original_db, timestamped_db_copy)

copied_db = copyfile(original_db, timestamped_db_copy)
copied_db

'data/intellectuels_copie2_20210704_194619.sqlite'

### birthyear (dbo)

#### 10000

In [8]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 22

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(22,
 'Cherche les années de naissances (dbo)',
 'Cherche les années de naissances (dbo)\r\n10000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT ?person ?birthYear\r\nWHERE\r\n  { ?person  dbo:birthYear  ?birthYear\r\n      { ?person  a                     yago:Intellectual109621545}\r\n  }\r\n ORDER BY ?person',
 None,
 '2021-06-17 13:33:09')

In [9]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [10]:
len(qr['results']['bindings'])

10000

In [12]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [13]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[["http://dbpedia.org/resource/'Abd_al-'Aziz_al-Wafa'i", '0811'],
 ["http://dbpedia.org/resource/A'Lelia_Bundles", '1952'],
 ['http://dbpedia.org/resource/A.G._Lafley', '1947'],
 ['http://dbpedia.org/resource/A.K._Nazmul_Karim', '1922'],
 ['http://dbpedia.org/resource/A._Alfred_Taubman', '1924']]

In [14]:
rl = [[l[0], 'http://dbpedia.org/ontology/birthYear', l[1], 'https://dbpedia.org/sparql', 1, 22 ] for l in r]

In [15]:
print(len(rl))
rl[:3]

10000


[["http://dbpedia.org/resource/'Abd_al-'Aziz_al-Wafa'i",
  'http://dbpedia.org/ontology/birthYear',
  '0811',
  'https://dbpedia.org/sparql',
  1,
  22],
 ["http://dbpedia.org/resource/A'Lelia_Bundles",
  'http://dbpedia.org/ontology/birthYear',
  '1952',
  'https://dbpedia.org/sparql',
  1,
  22],
 ['http://dbpedia.org/resource/A.G._Lafley',
  'http://dbpedia.org/ontology/birthYear',
  '1947',
  'https://dbpedia.org/sparql',
  1,
  22]]

In [17]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 20000

In [19]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 23

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(23,
 'Cherche les années de naissances (dbo)',
 'Cherche les années de naissances (dbo)\r\n20000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT ?person ?birthYear\r\nWHERE\r\n  { ?person  dbo:birthYear  ?birthYear\r\n      { ?person  a                     yago:Intellectual109621545}\r\n  }\r\nORDER BY ?person\r\nOFFSET 10000 LIMIT 10000\r\n',
 None,
 '2021-06-17 13:34:32')

In [20]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [21]:
len(qr['results']['bindings'])

10000

In [22]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [23]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Fred_Doucet', '1939'],
 ['http://dbpedia.org/resource/Fred_Dretske', '1932'],
 ['http://dbpedia.org/resource/Fred_Evans_(philosopher)', '1944'],
 ['http://dbpedia.org/resource/Fred_F._French', '1883'],
 ['http://dbpedia.org/resource/Fred_Feldman_(philosopher)', '1941']]

In [24]:
rl = [[l[0], 'http://dbpedia.org/ontology/birthYear', l[1], 'https://dbpedia.org/sparql', 1, 23 ] for l in r]

In [25]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Fred_Doucet',
  'http://dbpedia.org/ontology/birthYear',
  '1939',
  'https://dbpedia.org/sparql',
  1,
  23],
 ['http://dbpedia.org/resource/Fred_Dretske',
  'http://dbpedia.org/ontology/birthYear',
  '1932',
  'https://dbpedia.org/sparql',
  1,
  23],
 ['http://dbpedia.org/resource/Fred_Evans_(philosopher)',
  'http://dbpedia.org/ontology/birthYear',
  '1944',
  'https://dbpedia.org/sparql',
  1,
  23]]

In [26]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 30000

In [27]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 24

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(24,
 'Cherche les années de naissances (dbo)',
 'Cherche les années de naissances (dbo)\r\n30000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT ?person ?birthYear\r\nWHERE\r\n  { ?person  dbo:birthYear  ?birthYear\r\n      { ?person  a                     yago:Intellectual109621545}\r\n  }\r\nORDER BY ?person\r\nOFFSET 20000 LIMIT 10000\r\n',
 None,
 '2021-06-17 13:34:48')

In [28]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [29]:
len(qr['results']['bindings'])

10000

In [30]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [31]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Luisella_Beghi', '1922'],
 ['http://dbpedia.org/resource/Luka_Jones', '1975'],
 ['http://dbpedia.org/resource/Luke_Arnold', '1984'],
 ['http://dbpedia.org/resource/Luke_Brugnara', '1963'],
 ['http://dbpedia.org/resource/Luke_Chueh', '1973']]

In [32]:
rl = [[l[0], 'http://dbpedia.org/ontology/birthYear', l[1], 'https://dbpedia.org/sparql', 1, 24 ] for l in r]

In [33]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Luisella_Beghi',
  'http://dbpedia.org/ontology/birthYear',
  '1922',
  'https://dbpedia.org/sparql',
  1,
  24],
 ['http://dbpedia.org/resource/Luka_Jones',
  'http://dbpedia.org/ontology/birthYear',
  '1975',
  'https://dbpedia.org/sparql',
  1,
  24],
 ['http://dbpedia.org/resource/Luke_Arnold',
  'http://dbpedia.org/ontology/birthYear',
  '1984',
  'https://dbpedia.org/sparql',
  1,
  24]]

In [34]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 40000

In [35]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 25

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(25,
 'Cherche les années de naissances (dbo)',
 'Cherche les années de naissances (dbo)\r\n40000 (max)',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT ?person ?birthYear\r\nWHERE\r\n  { ?person  dbo:birthYear  ?birthYear\r\n      { ?person  a                     yago:Intellectual109621545}\r\n  }\r\nORDER BY ?person\r\nOFFSET 30000 LIMIT 10000\r\n',
 None,
 '2021-06-17 13:34:56')

In [36]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [37]:
len(qr['results']['bindings'])

3445

In [38]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [39]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

3445


[['http://dbpedia.org/resource/T._M._Nair', '1868'],
 ['http://dbpedia.org/resource/T._M._Scanlon', '1940'],
 ['http://dbpedia.org/resource/T._Muthuswamy_Iyer', '1832'],
 ['http://dbpedia.org/resource/T._Nadaraja', '1917'],
 ['http://dbpedia.org/resource/T._Norman_Mansell', '1904']]

In [40]:
rl = [[l[0], 'http://dbpedia.org/ontology/birthYear', l[1], 'https://dbpedia.org/sparql', 1, 25 ] for l in r]

In [41]:
print(len(rl))
rl[:3]

3445


[['http://dbpedia.org/resource/T._M._Nair',
  'http://dbpedia.org/ontology/birthYear',
  '1868',
  'https://dbpedia.org/sparql',
  1,
  25],
 ['http://dbpedia.org/resource/T._M._Scanlon',
  'http://dbpedia.org/ontology/birthYear',
  '1940',
  'https://dbpedia.org/sparql',
  1,
  25],
 ['http://dbpedia.org/resource/T._Muthuswamy_Iyer',
  'http://dbpedia.org/ontology/birthYear',
  '1832',
  'https://dbpedia.org/sparql',
  1,
  25]]

In [42]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### inspection

In [43]:
# connnexion à la base de données
cn = sql.connect(db)
c = cn.cursor()

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c.execute('SELECT count(*) AS eff, property FROM property GROUP BY property')
result_q = c.fetchall()

# fermer la connexion
cn.close()
result_q[:30]

[(33445, 'http://dbpedia.org/ontology/birthYear')]

La requête 46 donne le même résultat ! birthyear OK !

### birthdate (dbo)

#### 10000

In [44]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 26

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(26,
 'Cherche les dates de naissances (dbo)',
 'Cherche les dates de naissances (dbo)\r\n10000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT ?person ?birthDate\r\nWHERE\r\n  { ?person  dbo:birthDate  ?birthDate\r\n      { ?person  a                     yago:Intellectual109621545}\r\n  }\r\n ORDER BY ?person',
 None,
 '2021-06-17 13:51:31')

In [45]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [46]:
len(qr['results']['bindings'])

10000

In [47]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [48]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/9th_Wonder', '1975-01-15'],
 ["http://dbpedia.org/resource/A'Lelia_Bundles", '1952-06-07'],
 ['http://dbpedia.org/resource/A.B.C._Whipple', '1918-07-15'],
 ['http://dbpedia.org/resource/A.G._Lafley', '1947-06-13'],
 ['http://dbpedia.org/resource/A.J._Han_Vinck', '1949-05-15']]

In [49]:
rl = [[l[0], 'http://dbpedia.org/ontology/birthDate', l[1], 'https://dbpedia.org/sparql', 1, 26 ] for l in r]

In [50]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/9th_Wonder',
  'http://dbpedia.org/ontology/birthDate',
  '1975-01-15',
  'https://dbpedia.org/sparql',
  1,
  26],
 ["http://dbpedia.org/resource/A'Lelia_Bundles",
  'http://dbpedia.org/ontology/birthDate',
  '1952-06-07',
  'https://dbpedia.org/sparql',
  1,
  26],
 ['http://dbpedia.org/resource/A.B.C._Whipple',
  'http://dbpedia.org/ontology/birthDate',
  '1918-07-15',
  'https://dbpedia.org/sparql',
  1,
  26]]

In [51]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 20000

In [52]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 27

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(27,
 'Cherche les dates de naissances (dbo)',
 'Cherche les dates de naissances (dbo)\r\n20000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT ?person ?birthDate\r\nWHERE\r\n  { ?person  dbo:birthDate  ?birthDate\r\n      { ?person  a                     yago:Intellectual109621545}\r\n  }\r\nORDER BY ?person\r\nOFFSET 10000 LIMIT 10000\r\n',
 None,
 '2021-06-17 13:52:17')

In [53]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [54]:
len(qr['results']['bindings'])

10000

In [55]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [56]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Brian_Linighan', '1973-11-02'],
 ['http://dbpedia.org/resource/Brian_Loughnane', '1957-11-11'],
 ['http://dbpedia.org/resource/Brian_M._Fagan', '1936-08-01'],
 ['http://dbpedia.org/resource/Brian_MacArthur', '1940-02-05'],
 ['http://dbpedia.org/resource/Brian_MacLellan', '1958-10-27']]

In [57]:
rl = [[l[0], 'http://dbpedia.org/ontology/birthDate', l[1], 'https://dbpedia.org/sparql', 1, 27 ] for l in r]

In [58]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Brian_Linighan',
  'http://dbpedia.org/ontology/birthDate',
  '1973-11-02',
  'https://dbpedia.org/sparql',
  1,
  27],
 ['http://dbpedia.org/resource/Brian_Loughnane',
  'http://dbpedia.org/ontology/birthDate',
  '1957-11-11',
  'https://dbpedia.org/sparql',
  1,
  27],
 ['http://dbpedia.org/resource/Brian_M._Fagan',
  'http://dbpedia.org/ontology/birthDate',
  '1936-08-01',
  'https://dbpedia.org/sparql',
  1,
  27]]

In [59]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 30000

In [60]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 28

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(28,
 'Cherche les dates de naissances (dbo)',
 'Cherche les dates de naissances (dbo)\r\n30000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT ?person ?birthDate\r\nWHERE\r\n  { ?person  dbo:birthDate  ?birthDate\r\n      { ?person  a                     yago:Intellectual109621545}\r\n  }\r\nORDER BY ?person\r\nOFFSET 20000 LIMIT 10000\r\n',
 None,
 '2021-06-17 13:52:26')

In [61]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [62]:
len(qr['results']['bindings'])

10000

In [63]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [64]:
rl = [[l[0], 'http://dbpedia.org/ontology/birthDate', l[1], 'https://dbpedia.org/sparql', 1, 28 ] for l in r]

In [65]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Donny_George_Youkhanna',
  'http://dbpedia.org/ontology/birthDate',
  '1950-10-23',
  'https://dbpedia.org/sparql',
  1,
  28],
 ['http://dbpedia.org/resource/Donny_Harrel',
  'http://dbpedia.org/ontology/birthDate',
  '1969-12-17',
  'https://dbpedia.org/sparql',
  1,
  28],
 ['http://dbpedia.org/resource/Donny_Hathaway',
  'http://dbpedia.org/ontology/birthDate',
  '1945-10-01',
  'https://dbpedia.org/sparql',
  1,
  28]]

In [66]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 40000

In [67]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 29

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(29,
 'Cherche les dates de naissances (dbo)',
 'Cherche les dates de naissances (dbo)\r\n40000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT ?person ?birthDate\r\nWHERE\r\n  { ?person  dbo:birthDate  ?birthDate\r\n      { ?person  a                     yago:Intellectual109621545}\r\n  }\r\nORDER BY ?person\r\nOFFSET 30000 LIMIT 10000\r\n',
 None,
 '2021-06-17 13:52:39')

In [68]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [69]:
len(qr['results']['bindings'])

10000

In [70]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [71]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Gretchen_Albrecht', '1943-05-07'],
 ['http://dbpedia.org/resource/Gretchen_Barretto', '1970-03-06'],
 ['http://dbpedia.org/resource/Gretchen_Corbett', '1947-08-13'],
 ['http://dbpedia.org/resource/Gretchen_Cryer', '1935-10-17'],
 ['http://dbpedia.org/resource/Gretchen_Morgenson', '1956-01-02']]

In [72]:
rl = [[l[0], 'http://dbpedia.org/ontology/birthDate', l[1], 'https://dbpedia.org/sparql', 1, 29 ] for l in r]

In [73]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Gretchen_Albrecht',
  'http://dbpedia.org/ontology/birthDate',
  '1943-05-07',
  'https://dbpedia.org/sparql',
  1,
  29],
 ['http://dbpedia.org/resource/Gretchen_Barretto',
  'http://dbpedia.org/ontology/birthDate',
  '1970-03-06',
  'https://dbpedia.org/sparql',
  1,
  29],
 ['http://dbpedia.org/resource/Gretchen_Corbett',
  'http://dbpedia.org/ontology/birthDate',
  '1947-08-13',
  'https://dbpedia.org/sparql',
  1,
  29]]

In [74]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 50000

In [75]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 30

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(30,
 'Cherche les dates de naissances (dbo)',
 'Cherche les dates de naissances (dbo)\r\n50000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?person ?birthDate \r\n WHERE \r\n   {\r\n     {\r\n       SELECT DISTINCT ?person ?birthDate \r\nWHERE \r\n             { ?person  dbo:birthDate ?birthDate\r\n                {?person  a                     yago:Intellectual109621545}\r\n             } \r\n       ORDER BY ?person \r\n     }\r\n   } \r\nOFFSET 40000 LIMIT 10000',
 None,
 '2021-06-17 13:56:57')

In [76]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [77]:
len(qr['results']['bindings'])

10000

In [78]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [79]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Jiří_Bělohlávek', '1946-02-24'],
 ['http://dbpedia.org/resource/Jiří_Holík', '1944-07-09'],
 ['http://dbpedia.org/resource/Jiří_Stivín', '1942-11-23'],
 ['http://dbpedia.org/resource/Jiří_Traxler', '1912-03-12'],
 ['http://dbpedia.org/resource/Jiří_Šedivý', '1963-08-20']]

In [80]:
rl = [[l[0], 'http://dbpedia.org/ontology/birthDate', l[1], 'https://dbpedia.org/sparql', 1, 30 ] for l in r]

In [81]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Jiří_Bělohlávek',
  'http://dbpedia.org/ontology/birthDate',
  '1946-02-24',
  'https://dbpedia.org/sparql',
  1,
  30],
 ['http://dbpedia.org/resource/Jiří_Holík',
  'http://dbpedia.org/ontology/birthDate',
  '1944-07-09',
  'https://dbpedia.org/sparql',
  1,
  30],
 ['http://dbpedia.org/resource/Jiří_Stivín',
  'http://dbpedia.org/ontology/birthDate',
  '1942-11-23',
  'https://dbpedia.org/sparql',
  1,
  30]]

In [82]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 60000

In [83]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 31

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(31,
 'Cherche les dates de naissances (dbo)',
 'Cherche les dates de naissances (dbo)\r\n60000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?person ?birthDate \r\n WHERE \r\n   {\r\n     {\r\n       SELECT DISTINCT ?person ?birthDate \r\nWHERE \r\n             { ?person  dbo:birthDate ?birthDate\r\n                {?person  a                     yago:Intellectual109621545}\r\n             } \r\n       ORDER BY ?person \r\n     }\r\n   } \r\nOFFSET 50000 LIMIT 10000',
 None,
 '2021-06-17 13:57:04')

In [84]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [85]:
len(qr['results']['bindings'])

10000

In [86]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [87]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Lemuel_Williams', '1747-06-18'],
 ['http://dbpedia.org/resource/Len_Andrews', '1888-12-09'],
 ['http://dbpedia.org/resource/Len_Barnum', '1912-09-18'],
 ['http://dbpedia.org/resource/Len_Berman', '1947-06-14'],
 ['http://dbpedia.org/resource/Len_Deighton', '1929-02-18']]

In [88]:
rl = [[l[0], 'http://dbpedia.org/ontology/birthDate', l[1], 'https://dbpedia.org/sparql', 1, 31 ] for l in r]

In [89]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Lemuel_Williams',
  'http://dbpedia.org/ontology/birthDate',
  '1747-06-18',
  'https://dbpedia.org/sparql',
  1,
  31],
 ['http://dbpedia.org/resource/Len_Andrews',
  'http://dbpedia.org/ontology/birthDate',
  '1888-12-09',
  'https://dbpedia.org/sparql',
  1,
  31],
 ['http://dbpedia.org/resource/Len_Barnum',
  'http://dbpedia.org/ontology/birthDate',
  '1912-09-18',
  'https://dbpedia.org/sparql',
  1,
  31]]

In [90]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 70000

In [91]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 32

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(32,
 'Cherche les dates de naissances (dbo)',
 'Cherche les dates de naissances (dbo)\r\n70000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?person ?birthDate \r\n WHERE \r\n   {\r\n     {\r\n       SELECT DISTINCT ?person ?birthDate \r\nWHERE \r\n             { ?person  dbo:birthDate ?birthDate\r\n                {?person  a                     yago:Intellectual109621545}\r\n             } \r\n       ORDER BY ?person \r\n     }\r\n   } \r\nOFFSET 60000 LIMIT 10000',
 None,
 '2021-06-17 13:57:26')

In [92]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [93]:
len(qr['results']['bindings'])

10000

In [94]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [95]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Neil_McPherson_(artistic_director)',
  '1969-10-07'],
 ['http://dbpedia.org/resource/Neil_Megson_(soccer)', '1962-07-27'],
 ['http://dbpedia.org/resource/Neil_Morrissey', '1962-07-04'],
 ["http://dbpedia.org/resource/Neil_O'Brien", '1978-11-06'],
 ["http://dbpedia.org/resource/Neil_O'Leary", '1958-10-10']]

In [96]:
rl = [[l[0], 'http://dbpedia.org/ontology/birthDate', l[1], 'https://dbpedia.org/sparql', 1, 32 ] for l in r]

In [97]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Neil_McPherson_(artistic_director)',
  'http://dbpedia.org/ontology/birthDate',
  '1969-10-07',
  'https://dbpedia.org/sparql',
  1,
  32],
 ['http://dbpedia.org/resource/Neil_Megson_(soccer)',
  'http://dbpedia.org/ontology/birthDate',
  '1962-07-27',
  'https://dbpedia.org/sparql',
  1,
  32],
 ['http://dbpedia.org/resource/Neil_Morrissey',
  'http://dbpedia.org/ontology/birthDate',
  '1962-07-04',
  'https://dbpedia.org/sparql',
  1,
  32]]

In [98]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 80000

In [99]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 33

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(33,
 'Cherche les dates de naissances (dbo)',
 'Cherche les dates de naissances (dbo)\r\n80000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?person ?birthDate \r\n WHERE \r\n   {\r\n     {\r\n       SELECT DISTINCT ?person ?birthDate \r\nWHERE \r\n             { ?person  dbo:birthDate ?birthDate\r\n                {?person  a                     yago:Intellectual109621545}\r\n             } \r\n       ORDER BY ?person \r\n     }\r\n   } \r\nOFFSET 70000 LIMIT 10000',
 None,
 '2021-06-17 13:57:42')

In [100]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [101]:
len(qr['results']['bindings'])

10000

In [102]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [103]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Roberto_Bisacco', '1938-03-01'],
 ['http://dbpedia.org/resource/Roberto_Bruce', '1979-07-30'],
 ['http://dbpedia.org/resource/Roberto_Calasso', '1941-05-30'],
 ['http://dbpedia.org/resource/Roberto_Cassá', '1948-09-12'],
 ['http://dbpedia.org/resource/Roberto_Castelli', '1946-07-12']]

In [104]:
rl = [[l[0], 'http://dbpedia.org/ontology/birthDate', l[1], 'https://dbpedia.org/sparql', 1, 33 ] for l in r]

In [105]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Roberto_Bisacco',
  'http://dbpedia.org/ontology/birthDate',
  '1938-03-01',
  'https://dbpedia.org/sparql',
  1,
  33],
 ['http://dbpedia.org/resource/Roberto_Bruce',
  'http://dbpedia.org/ontology/birthDate',
  '1979-07-30',
  'https://dbpedia.org/sparql',
  1,
  33],
 ['http://dbpedia.org/resource/Roberto_Calasso',
  'http://dbpedia.org/ontology/birthDate',
  '1941-05-30',
  'https://dbpedia.org/sparql',
  1,
  33]]

In [106]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 90000

In [107]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 34

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(34,
 'Cherche les dates de naissances (dbo)',
 'Cherche les dates de naissances (dbo)\r\n90000 (max)',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?person ?birthDate\r\n WHERE \r\n   {\r\n     {\r\n       SELECT DISTINCT ?person ?birthDate \r\nWHERE \r\n             { ?person  dbo:birthDate ?birthDate\r\n                {?person  a                     yago:Intellectual109621545}\r\n             } \r\n       ORDER BY ?person \r\n     }\r\n   } \r\nOFFSET 80000 LIMIT 10000',
 None,
 '2021-06-17 13:57:49')

In [108]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [109]:
len(qr['results']['bindings'])

6270

In [110]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [111]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

6270


[['http://dbpedia.org/resource/Tomoyasu_Ando', '1974-05-23'],
 ['http://dbpedia.org/resource/Tomoyoshi_Ikeya', '1962-06-17'],
 ['http://dbpedia.org/resource/Tomoyuki_Arata', '1985-10-03'],
 ['http://dbpedia.org/resource/Tomoyuki_Furumaya', '1968-11-14'],
 ['http://dbpedia.org/resource/Tomás_Balduino', '1922-12-31']]

In [112]:
rl = [[l[0], 'http://dbpedia.org/ontology/birthDate', l[1], 'https://dbpedia.org/sparql', 1, 34 ] for l in r]

In [113]:
print(len(rl))
rl[:3]

6270


[['http://dbpedia.org/resource/Tomoyasu_Ando',
  'http://dbpedia.org/ontology/birthDate',
  '1974-05-23',
  'https://dbpedia.org/sparql',
  1,
  34],
 ['http://dbpedia.org/resource/Tomoyoshi_Ikeya',
  'http://dbpedia.org/ontology/birthDate',
  '1962-06-17',
  'https://dbpedia.org/sparql',
  1,
  34],
 ['http://dbpedia.org/resource/Tomoyuki_Arata',
  'http://dbpedia.org/ontology/birthDate',
  '1985-10-03',
  'https://dbpedia.org/sparql',
  1,
  34]]

In [114]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

### birthdate (dbp)

#### 10000

In [116]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 35

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(35,
 'Cherche les dates de naissances (dbp)',
 'Cherche les dates de naissances (dbp)\r\n10000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT ?person ?birthDate\r\nWHERE\r\n  { ?person  dbp:birthDate  ?birthDate\r\n      { ?person  a                     yago:Intellectual109621545}\r\n  }\r\n ORDER BY ?person',
 None,
 '2021-07-04 17:13:42')

In [117]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [118]:
len(qr['results']['bindings'])

10000

In [119]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [120]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[["http://dbpedia.org/resource/'Abd_al-'Aziz_al-Wafa'i", '811'],
 ['http://dbpedia.org/resource/14th_Dalai_Lama', '1935-07-06'],
 ['http://dbpedia.org/resource/5th_Dalai_Lama', '1617'],
 ['http://dbpedia.org/resource/9th_Wonder', '1975-01-15'],
 ["http://dbpedia.org/resource/A'Lelia_Bundles", '1952-06-07']]

In [121]:
rl = [[l[0], 'http://dbpedia.org/property/birthDate', l[1], 'https://dbpedia.org/sparql', 1, 35 ] for l in r]

print(len(rl))
rl[:3]

In [123]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 20000

In [124]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 36

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(36,
 'Cherche les dates de naissances (dbp)',
 'Cherche les dates de naissances (dbp)\r\n20000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT ?person ?birthDate\r\nWHERE\r\n  { ?person  dbp:birthDate  ?birthDate\r\n      { ?person  a                     yago:Intellectual109621545}\r\n  }\r\n ORDER BY ?person\r\n OFFSET 10000 LIMIT 10000\r\n ',
 None,
 '2021-07-04 19:19:54')

In [125]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [126]:
len(qr['results']['bindings'])

10000

In [127]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [128]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Bertram_Heyn', '1912-10-01'],
 ['http://dbpedia.org/resource/Bertram_J._Rodgers', '1894-03-18'],
 ['http://dbpedia.org/resource/Bertram_Kostant', '1928-05-24'],
 ['http://dbpedia.org/resource/Bertram_L._Podell', '1925-12-27'],
 ['http://dbpedia.org/resource/Bertram_Raphael', '1936-11-16']]

In [129]:
rl = [[l[0], 'http://dbpedia.org/property/birthDate', l[1], 'https://dbpedia.org/sparql', 1, 36 ] for l in r]

In [130]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Bertram_Heyn',
  'http://dbpedia.org/property/birthDate',
  '1912-10-01',
  'https://dbpedia.org/sparql',
  1,
  36],
 ['http://dbpedia.org/resource/Bertram_J._Rodgers',
  'http://dbpedia.org/property/birthDate',
  '1894-03-18',
  'https://dbpedia.org/sparql',
  1,
  36],
 ['http://dbpedia.org/resource/Bertram_Kostant',
  'http://dbpedia.org/property/birthDate',
  '1928-05-24',
  'https://dbpedia.org/sparql',
  1,
  36]]

In [131]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 30000

In [132]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 37

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(37,
 'Cherche les dates de naissances (dbp)',
 'Cherche les dates de naissances (dbp)\r\n30000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT ?person ?birthDate\r\nWHERE\r\n  { ?person  dbp:birthDate  ?birthDate\r\n      { ?person  a                     yago:Intellectual109621545}\r\n  }\r\n ORDER BY ?person\r\n OFFSET 20000 LIMIT 10000\r\n ',
 None,
 '2021-07-04 19:19:54')

In [133]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [134]:
len(qr['results']['bindings'])

10000

In [135]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [136]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Dave_Price', '1966-10-18'],
 ['http://dbpedia.org/resource/Dave_Pureifory', '1949-07-12'],
 ['http://dbpedia.org/resource/Dave_Racaniello', '1978-06-03'],
 ['http://dbpedia.org/resource/Dave_Raggett', '1955-06-08'],
 ['http://dbpedia.org/resource/Dave_Ragone', '1979-10-03']]

In [137]:
rl = [[l[0], 'http://dbpedia.org/property/birthDate', l[1], 'https://dbpedia.org/sparql', 1, 37 ] for l in r]

In [138]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Dave_Price',
  'http://dbpedia.org/property/birthDate',
  '1966-10-18',
  'https://dbpedia.org/sparql',
  1,
  37],
 ['http://dbpedia.org/resource/Dave_Pureifory',
  'http://dbpedia.org/property/birthDate',
  '1949-07-12',
  'https://dbpedia.org/sparql',
  1,
  37],
 ['http://dbpedia.org/resource/Dave_Racaniello',
  'http://dbpedia.org/property/birthDate',
  '1978-06-03',
  'https://dbpedia.org/sparql',
  1,
  37]]

In [139]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 40000

In [140]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 38

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(38,
 'Cherche les dates de naissances (dbp)',
 'Cherche les dates de naissances (dbp)\r\n40000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT ?person ?birthDate\r\nWHERE\r\n  { ?person  dbp:birthDate  ?birthDate\r\n      { ?person  a                     yago:Intellectual109621545}\r\n  }\r\n ORDER BY ?person\r\n OFFSET 30000 LIMIT 10000\r\n ',
 None,
 '2021-07-04 19:19:55')

In [141]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [142]:
len(qr['results']['bindings'])

10000

In [143]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [144]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Frank_Drew', '1930-10-10'],
 ['http://dbpedia.org/resource/Frank_Driggs', '1930-01-29'],
 ['http://dbpedia.org/resource/Frank_DuMond', '1865-08-20'],
 ['http://dbpedia.org/resource/Frank_Duckworth', '1939-12-26'],
 ['http://dbpedia.org/resource/Frank_Dunlop_(director)', '1927-02-15']]

In [145]:
rl = [[l[0], 'http://dbpedia.org/property/birthDate', l[1], 'https://dbpedia.org/sparql', 1, 38 ] for l in r]

In [146]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Frank_Drew',
  'http://dbpedia.org/property/birthDate',
  '1930-10-10',
  'https://dbpedia.org/sparql',
  1,
  38],
 ['http://dbpedia.org/resource/Frank_Driggs',
  'http://dbpedia.org/property/birthDate',
  '1930-01-29',
  'https://dbpedia.org/sparql',
  1,
  38],
 ['http://dbpedia.org/resource/Frank_DuMond',
  'http://dbpedia.org/property/birthDate',
  '1865-08-20',
  'https://dbpedia.org/sparql',
  1,
  38]]

In [147]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 50000

In [148]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 39

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(39,
 'Cherche les dates de naissances (dbp)',
 'Cherche les dates de naissances (dbp)\r\n50000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?person ?birthDate \r\n WHERE \r\n   {\r\n     {\r\n       SELECT DISTINCT ?person ?birthDate \r\nWHERE \r\n             { ?person  dbp:birthDate ?birthDate\r\n                {?person  a                     yago:Intellectual109621545}\r\n             } \r\n       ORDER BY ?person \r\n     }\r\n   } \r\nOFFSET 40000 LIMIT 10000',
 None,
 '2021-07-04 19:27:39')

In [149]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [150]:
len(qr['results']['bindings'])

10000

In [151]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [152]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Huang_Xiaoming', '1977-11-13'],
 ['http://dbpedia.org/resource/Huang_Zongxi', '1610-09-24'],
 ['http://dbpedia.org/resource/Huang_Zunxian', '1848-05-29'],
 ['http://dbpedia.org/resource/Huang_Zuolin', '1906-10-24'],
 ['http://dbpedia.org/resource/Hub_Arkush', '1953-02-14']]

In [153]:
rl = [[l[0], 'http://dbpedia.org/property/birthDate', l[1], 'https://dbpedia.org/sparql', 1, 39 ] for l in r]

In [154]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Huang_Xiaoming',
  'http://dbpedia.org/property/birthDate',
  '1977-11-13',
  'https://dbpedia.org/sparql',
  1,
  39],
 ['http://dbpedia.org/resource/Huang_Zongxi',
  'http://dbpedia.org/property/birthDate',
  '1610-09-24',
  'https://dbpedia.org/sparql',
  1,
  39],
 ['http://dbpedia.org/resource/Huang_Zunxian',
  'http://dbpedia.org/property/birthDate',
  '1848-05-29',
  'https://dbpedia.org/sparql',
  1,
  39]]

In [155]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 60000

In [156]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 40

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(40,
 'Cherche les dates de naissances (dbp)',
 'Cherche les dates de naissances (dbp)\r\n60000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?person ?birthDate \r\n WHERE \r\n   {\r\n     {\r\n       SELECT DISTINCT ?person ?birthDate \r\nWHERE \r\n             { ?person  dbp:birthDate ?birthDate\r\n                {?person  a                     yago:Intellectual109621545}\r\n             } \r\n       ORDER BY ?person \r\n     }\r\n   } \r\nOFFSET 50000 LIMIT 10000',
 None,
 '2021-07-04 19:29:29')

In [157]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [158]:
len(qr['results']['bindings'])

10000

In [159]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [160]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/John_Harold_Owen_Wilsey', '1904'],
 ['http://dbpedia.org/resource/John_Harper_(politician)', '1930-05-03'],
 ['http://dbpedia.org/resource/John_Harris_(bioethicist)', '1945-08-21'],
 ['http://dbpedia.org/resource/John_Harris_(curator)', '1931'],
 ['http://dbpedia.org/resource/John_Harris_Baker', '1832-02-28']]

In [161]:
rl = [[l[0], 'http://dbpedia.org/property/birthDate', l[1], 'https://dbpedia.org/sparql', 1, 40 ] for l in r]

In [162]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/John_Harold_Owen_Wilsey',
  'http://dbpedia.org/property/birthDate',
  '1904',
  'https://dbpedia.org/sparql',
  1,
  40],
 ['http://dbpedia.org/resource/John_Harper_(politician)',
  'http://dbpedia.org/property/birthDate',
  '1930-05-03',
  'https://dbpedia.org/sparql',
  1,
  40],
 ['http://dbpedia.org/resource/John_Harris_(bioethicist)',
  'http://dbpedia.org/property/birthDate',
  '1945-08-21',
  'https://dbpedia.org/sparql',
  1,
  40]]

In [163]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 70000

In [164]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 41

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(41,
 'Cherche les dates de naissances (dbp)',
 'Cherche les dates de naissances (dbp)\r\n70000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?person ?birthDate \r\n WHERE \r\n   {\r\n     {\r\n       SELECT DISTINCT ?person ?birthDate \r\nWHERE \r\n             { ?person  dbp:birthDate ?birthDate\r\n                {?person  a                     yago:Intellectual109621545}\r\n             } \r\n       ORDER BY ?person \r\n     }\r\n   } \r\nOFFSET 60000 LIMIT 10000',
 None,
 '2021-07-04 19:29:29')

In [165]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [166]:
len(qr['results']['bindings'])

10000

In [167]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [168]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Lewis_Linn_McArthur', '1843-03-18'],
 ['http://dbpedia.org/resource/Lewis_M._Branscomb', '1926-08-17'],
 ['http://dbpedia.org/resource/Lewis_M._Haupt', '1844-03-21'],
 ['http://dbpedia.org/resource/Lewis_M._Long', '1883-06-22'],
 ['http://dbpedia.org/resource/Lewis_MacAdams', '1944-10-12']]

In [169]:
rl = [[l[0], 'http://dbpedia.org/property/birthDate', l[1], 'https://dbpedia.org/sparql', 1, 41 ] for l in r]

In [170]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Lewis_Linn_McArthur',
  'http://dbpedia.org/property/birthDate',
  '1843-03-18',
  'https://dbpedia.org/sparql',
  1,
  41],
 ['http://dbpedia.org/resource/Lewis_M._Branscomb',
  'http://dbpedia.org/property/birthDate',
  '1926-08-17',
  'https://dbpedia.org/sparql',
  1,
  41],
 ['http://dbpedia.org/resource/Lewis_M._Haupt',
  'http://dbpedia.org/property/birthDate',
  '1844-03-21',
  'https://dbpedia.org/sparql',
  1,
  41]]

In [171]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 80000

In [172]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 42

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(42,
 'Cherche les dates de naissances (dbp)',
 'Cherche les dates de naissances (dbp)\r\n80000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?person ?birthDate \r\n WHERE \r\n   {\r\n     {\r\n       SELECT DISTINCT ?person ?birthDate \r\nWHERE \r\n             { ?person  dbp:birthDate ?birthDate\r\n                {?person  a                     yago:Intellectual109621545}\r\n             } \r\n       ORDER BY ?person \r\n     }\r\n   } \r\nOFFSET 70000 LIMIT 10000',
 None,
 '2021-07-04 19:29:29')

In [173]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [174]:
len(qr['results']['bindings'])

10000

In [175]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [176]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Myles_Coverdale', 'c. 1488'],
 ['http://dbpedia.org/resource/Myles_Jackson', '1964-11-25'],
 ['http://dbpedia.org/resource/Myles_Lane', '1903-10-02'],
 ['http://dbpedia.org/resource/Myles_Lee', '1953'],
 ['http://dbpedia.org/resource/Myles_Munroe', '1954-04-20']]

In [177]:
rl = [[l[0], 'http://dbpedia.org/property/birthDate', l[1], 'https://dbpedia.org/sparql', 1, 42 ] for l in r]

In [178]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Myles_Coverdale',
  'http://dbpedia.org/property/birthDate',
  'c. 1488',
  'https://dbpedia.org/sparql',
  1,
  42],
 ['http://dbpedia.org/resource/Myles_Jackson',
  'http://dbpedia.org/property/birthDate',
  '1964-11-25',
  'https://dbpedia.org/sparql',
  1,
  42],
 ['http://dbpedia.org/resource/Myles_Lane',
  'http://dbpedia.org/property/birthDate',
  '1903-10-02',
  'https://dbpedia.org/sparql',
  1,
  42]]

In [179]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 90000

In [180]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 43

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(43,
 'Cherche les dates de naissances (dbp)',
 'Cherche les dates de naissances (dbp)\r\n90000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?person ?birthDate \r\n WHERE \r\n   {\r\n     {\r\n       SELECT DISTINCT ?person ?birthDate \r\nWHERE \r\n             { ?person  dbp:birthDate ?birthDate\r\n                {?person  a                     yago:Intellectual109621545}\r\n             } \r\n       ORDER BY ?person \r\n     }\r\n   } \r\nOFFSET 80000 LIMIT 10000',
 None,
 '2021-07-04 19:29:29')

In [181]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [182]:
len(qr['results']['bindings'])

10000

In [183]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [184]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Richard_Doyle_(actor)', '1945-04-26'],
 ['http://dbpedia.org/resource/Richard_Doyle_(author)', '1948-01-10'],
 ['http://dbpedia.org/resource/Richard_Drayton', '1964'],
 ['http://dbpedia.org/resource/Richard_Drouin', '1932-04-08'],
 ['http://dbpedia.org/resource/Richard_Duncan_(athlete)', '1973-12-25']]

In [185]:
rl = [[l[0], 'http://dbpedia.org/property/birthDate', l[1], 'https://dbpedia.org/sparql', 1, 43 ] for l in r]

In [186]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Richard_Doyle_(actor)',
  'http://dbpedia.org/property/birthDate',
  '1945-04-26',
  'https://dbpedia.org/sparql',
  1,
  43],
 ['http://dbpedia.org/resource/Richard_Doyle_(author)',
  'http://dbpedia.org/property/birthDate',
  '1948-01-10',
  'https://dbpedia.org/sparql',
  1,
  43],
 ['http://dbpedia.org/resource/Richard_Drayton',
  'http://dbpedia.org/property/birthDate',
  '1964',
  'https://dbpedia.org/sparql',
  1,
  43]]

In [187]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 100000

In [188]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 44

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(44,
 'Cherche les dates de naissances (dbp)',
 'Cherche les dates de naissances (dbp)\r\n100000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?person ?birthDate \r\n WHERE \r\n   {\r\n     {\r\n       SELECT DISTINCT ?person ?birthDate \r\nWHERE \r\n             { ?person  dbp:birthDate ?birthDate\r\n                {?person  a                     yago:Intellectual109621545}\r\n             } \r\n       ORDER BY ?person \r\n     }\r\n   } \r\nOFFSET 90000 LIMIT 10000',
 None,
 None)

In [189]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [190]:
len(qr['results']['bindings'])

10000

In [191]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [192]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Steve_Paikin', '1960-06-09'],
 ['http://dbpedia.org/resource/Steve_Pajcic', '1946-08-04'],
 ['http://dbpedia.org/resource/Steve_Parry_(musician)', '1958'],
 ['http://dbpedia.org/resource/Steve_Pearce_(baseball)', '1983-04-13'],
 ['http://dbpedia.org/resource/Steve_Pemberton', '1967-09-01']]

In [193]:
rl = [[l[0], 'http://dbpedia.org/property/birthDate', l[1], 'https://dbpedia.org/sparql', 1, 44 ] for l in r]

In [194]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Steve_Paikin',
  'http://dbpedia.org/property/birthDate',
  '1960-06-09',
  'https://dbpedia.org/sparql',
  1,
  44],
 ['http://dbpedia.org/resource/Steve_Pajcic',
  'http://dbpedia.org/property/birthDate',
  '1946-08-04',
  'https://dbpedia.org/sparql',
  1,
  44],
 ['http://dbpedia.org/resource/Steve_Parry_(musician)',
  'http://dbpedia.org/property/birthDate',
  '1958',
  'https://dbpedia.org/sparql',
  1,
  44]]

In [195]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 110000

In [196]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 45

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(45,
 'Cherche les dates de naissances (dbp)',
 'Cherche les dates de naissances (dbp)\r\n110000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?person ?birthDate \r\n WHERE \r\n   {\r\n     {\r\n       SELECT DISTINCT ?person ?birthDate \r\nWHERE \r\n             { ?person  dbp:birthDate ?birthDate\r\n                {?person  a                     yago:Intellectual109621545}\r\n             } \r\n       ORDER BY ?person \r\n     }\r\n   } \r\nOFFSET 100000 LIMIT 10000',
 None,
 '2021-07-04 19:30:13')

In [197]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [198]:
len(qr['results']['bindings'])

2170

In [199]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [200]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

2170


[['http://dbpedia.org/resource/William_Scully_(bishop)', '1894-08-06'],
 ['http://dbpedia.org/resource/William_Seager_(philosopher)', '1952-04-11'],
 ['http://dbpedia.org/resource/William_Seale', '1939'],
 ['http://dbpedia.org/resource/William_Sealy_Gosset', '1876-06-13'],
 ['http://dbpedia.org/resource/William_Seaman_Bainbridge', '1870-02-17']]

In [201]:
rl = [[l[0], 'http://dbpedia.org/property/birthDate', l[1], 'https://dbpedia.org/sparql', 1, 45 ] for l in r]

In [202]:
print(len(rl))
rl[:3]

2170


[['http://dbpedia.org/resource/William_Scully_(bishop)',
  'http://dbpedia.org/property/birthDate',
  '1894-08-06',
  'https://dbpedia.org/sparql',
  1,
  45],
 ['http://dbpedia.org/resource/William_Seager_(philosopher)',
  'http://dbpedia.org/property/birthDate',
  '1952-04-11',
  'https://dbpedia.org/sparql',
  1,
  45],
 ['http://dbpedia.org/resource/William_Seale',
  'http://dbpedia.org/property/birthDate',
  '1939',
  'https://dbpedia.org/sparql',
  1,
  45]]

In [203]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### inspection

In [204]:
# connnexion à la base de données
cn = sql.connect(db)
c = cn.cursor()

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c.execute('SELECT count(*) AS eff, property FROM property GROUP BY property')
result_q = c.fetchall()

# fermer la connexion
cn.close()
result_q[:30]

[(86270, 'http://dbpedia.org/ontology/birthDate'),
 (33445, 'http://dbpedia.org/ontology/birthYear'),
 (102170, 'http://dbpedia.org/property/birthDate')]

### Sauvegarde DB

Copie3 = entités person enregistrées / requêtes birthyear, birthdate ajoutées / propriétés birthyear (dbo) birthdate (dbo) et birthdate (dbp) ajoutées

In [205]:
### Fonction qui récupère et met en forme le temps au moment de l'exécution

# définition
def timestamp_formatted_for_file_name():
    is_now = time.strftime('%Y%m%d_%H%M%S')
    return is_now

# exécution
timestamp_formatted_for_file_name()

'20210704_230734'

In [206]:
### Définir les adresses des fichiers, l'existant et celui à créer
original_db = 'data/intellectuels.db'

##  Noter que la différence de suffixe, en soi totalement arbitraire, 
#  dépend du fait que dans la configuration de .gitignore, .sqlite est exclu du verisonnement GIT
#  contrairement à .db qui est versionné
timestamped_db_copy = 'data/intellectuels_copie3_' + timestamp_formatted_for_file_name() + '.sqlite'

In [207]:
### Créer une copie de sauvegarde avec timestamp du fichier (versionnement manuel)
# ATTENTION : la base de données doit exister à l'endroit indiqué !
## Cette requête n'est utile que si des modifications en écriture vont être apportées à la base de données,
# afin de préserver le dernier état avant modification

## Documentation:
# https://docs.python.org/3/library/shutil.htmlcopied_db = copyfile(original_db, timestamped_db_copy)

copied_db = copyfile(original_db, timestamped_db_copy)
copied_db

'data/intellectuels_copie3_20210704_230745.sqlite'

### occupation (dbo)

#### 10000

In [208]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 49

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(49,
 'Cherche les occupation (dbo)',
 'Cherche les occupation (dbo)\r\n10000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT ?person ?occupation\r\nWHERE\r\n  { ?person  dbo:occupation  ?occupation\r\n      { ?person  a                     yago:Intellectual109621545}\r\n  }\r\n ORDER BY ?person',
 None,
 '2021-07-04 23:53:41')

In [209]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [210]:
len(qr['results']['bindings'])

10000

In [211]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [212]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/9th_Wonder',
  'http://dbpedia.org/resource/9th_Wonder_production_discography'],
 ['http://dbpedia.org/resource/9th_Wonder',
  'http://dbpedia.org/resource/Rapping'],
 ['http://dbpedia.org/resource/9th_Wonder',
  'http://dbpedia.org/resource/Record_executive'],
 ["http://dbpedia.org/resource/A'Lelia_Bundles",
  "http://dbpedia.org/resource/A'Lelia_Bundles__PersonFunction__1"],
 ['http://dbpedia.org/resource/A.B.C._Whipple',
  'http://dbpedia.org/resource/Historian']]

In [213]:
rl = [[l[0], 'http://dbpedia.org/ontology/occupation', l[1], 'https://dbpedia.org/sparql', 1, 49 ] for l in r]

In [214]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/9th_Wonder',
  'http://dbpedia.org/ontology/occupation',
  'http://dbpedia.org/resource/9th_Wonder_production_discography',
  'https://dbpedia.org/sparql',
  1,
  49],
 ['http://dbpedia.org/resource/9th_Wonder',
  'http://dbpedia.org/ontology/occupation',
  'http://dbpedia.org/resource/Rapping',
  'https://dbpedia.org/sparql',
  1,
  49],
 ['http://dbpedia.org/resource/9th_Wonder',
  'http://dbpedia.org/ontology/occupation',
  'http://dbpedia.org/resource/Record_executive',
  'https://dbpedia.org/sparql',
  1,
  49]]

In [215]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 20000

In [216]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 50

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(50,
 'Cherche les occupation (dbo)',
 'Cherche les occupation (dbo)\r\n20000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT ?person ?occupation\r\nWHERE\r\n  { ?person  dbo:occupation  ?occupation\r\n      { ?person  a                     yago:Intellectual109621545}\r\n  }\r\n ORDER BY ?person\r\n OFFSET 10000 LIMIT 10000',
 None,
 '2021-07-05 00:11:16')

In [217]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [218]:
len(qr['results']['bindings'])

10000

In [219]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [220]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Christopher_Mellon',
  'http://dbpedia.org/resource/Deputy_Assistant_Secretary_of_Defense'],
 ['http://dbpedia.org/resource/Christopher_Mellon',
  'http://dbpedia.org/resource/Christopher_Mellon__PersonFunction__1'],
 ['http://dbpedia.org/resource/Christopher_Merrill',
  'http://dbpedia.org/resource/University_of_Iowa'],
 ['http://dbpedia.org/resource/Christopher_Merrill',
  'http://dbpedia.org/resource/International_Writing_Program'],
 ['http://dbpedia.org/resource/Christopher_Moore_(author)',
  'http://dbpedia.org/resource/Novelist']]

In [221]:
rl = [[l[0], 'http://dbpedia.org/ontology/occupation', l[1], 'https://dbpedia.org/sparql', 1, 50 ] for l in r]

In [222]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Christopher_Mellon',
  'http://dbpedia.org/ontology/occupation',
  'http://dbpedia.org/resource/Deputy_Assistant_Secretary_of_Defense',
  'https://dbpedia.org/sparql',
  1,
  50],
 ['http://dbpedia.org/resource/Christopher_Mellon',
  'http://dbpedia.org/ontology/occupation',
  'http://dbpedia.org/resource/Christopher_Mellon__PersonFunction__1',
  'https://dbpedia.org/sparql',
  1,
  50],
 ['http://dbpedia.org/resource/Christopher_Merrill',
  'http://dbpedia.org/ontology/occupation',
  'http://dbpedia.org/resource/University_of_Iowa',
  'https://dbpedia.org/sparql',
  1,
  50]]

In [223]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 30000

In [224]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 51

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(51,
 'Cherche les occupation (dbo)',
 'Cherche les occupation (dbo)\r\n30000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT ?person ?occupation\r\nWHERE\r\n  { ?person  dbo:occupation  ?occupation\r\n      { ?person  a                     yago:Intellectual109621545}\r\n  }\r\n ORDER BY ?person\r\n OFFSET 20000 LIMIT 10000',
 None,
 '2021-07-05 00:11:16')

In [225]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [226]:
len(qr['results']['bindings'])

10000

In [227]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [228]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Gerry_Campbell',
  'http://dbpedia.org/resource/Executive_officer'],
 ['http://dbpedia.org/resource/Gerry_Campbell',
  'http://dbpedia.org/resource/Entrepreneur'],
 ['http://dbpedia.org/resource/Gerry_Campbell',
  'http://dbpedia.org/resource/Gerry_Campbell__PersonFunction__1'],
 ['http://dbpedia.org/resource/Gerry_Connolly_(comedian)',
  'http://dbpedia.org/resource/Actor'],
 ['http://dbpedia.org/resource/Gerry_Connolly_(comedian)',
  'http://dbpedia.org/resource/Gerry_Connolly_(comedian)__PersonFunction__1']]

In [229]:
rl = [[l[0], 'http://dbpedia.org/ontology/occupation', l[1], 'https://dbpedia.org/sparql', 1, 51 ] for l in r]

In [230]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Gerry_Campbell',
  'http://dbpedia.org/ontology/occupation',
  'http://dbpedia.org/resource/Executive_officer',
  'https://dbpedia.org/sparql',
  1,
  51],
 ['http://dbpedia.org/resource/Gerry_Campbell',
  'http://dbpedia.org/ontology/occupation',
  'http://dbpedia.org/resource/Entrepreneur',
  'https://dbpedia.org/sparql',
  1,
  51],
 ['http://dbpedia.org/resource/Gerry_Campbell',
  'http://dbpedia.org/ontology/occupation',
  'http://dbpedia.org/resource/Gerry_Campbell__PersonFunction__1',
  'https://dbpedia.org/sparql',
  1,
  51]]

In [231]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 40000

In [232]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 52

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(52,
 'Cherche les occupation (dbo)',
 'Cherche les occupation (dbo)\r\n40000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT ?person ?occupation\r\nWHERE\r\n  { ?person  dbo:occupation  ?occupation\r\n      { ?person  a                     yago:Intellectual109621545}\r\n  }\r\n ORDER BY ?person\r\n OFFSET 30000 LIMIT 10000',
 None,
 '2021-07-05 00:11:16')

In [233]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [234]:
len(qr['results']['bindings'])

10000

In [235]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [236]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/John_Wheelwright',
  'http://dbpedia.org/resource/John_Wheelwright__PersonFunction__1'],
 ['http://dbpedia.org/resource/John_Whelan_Sterling',
  'http://dbpedia.org/resource/John_Whelan_Sterling__PersonFunction__1'],
 ['http://dbpedia.org/resource/John_White,_1st_Baron_Overtoun',
  'http://dbpedia.org/resource/John_White,_1st_Baron_Overtoun__PersonFunction__1'],
 ['http://dbpedia.org/resource/John_White_(composer)',
  'http://dbpedia.org/resource/Composer'],
 ['http://dbpedia.org/resource/John_Whitelaw_(general,_born_1894)',
  'http://dbpedia.org/resource/Royal_Regiment_of_Australian_Artillery']]

In [237]:
rl = [[l[0], 'http://dbpedia.org/ontology/occupation', l[1], 'https://dbpedia.org/sparql', 1, 52 ] for l in r]

In [238]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/John_Wheelwright',
  'http://dbpedia.org/ontology/occupation',
  'http://dbpedia.org/resource/John_Wheelwright__PersonFunction__1',
  'https://dbpedia.org/sparql',
  1,
  52],
 ['http://dbpedia.org/resource/John_Whelan_Sterling',
  'http://dbpedia.org/ontology/occupation',
  'http://dbpedia.org/resource/John_Whelan_Sterling__PersonFunction__1',
  'https://dbpedia.org/sparql',
  1,
  52],
 ['http://dbpedia.org/resource/John_White,_1st_Baron_Overtoun',
  'http://dbpedia.org/ontology/occupation',
  'http://dbpedia.org/resource/John_White,_1st_Baron_Overtoun__PersonFunction__1',
  'https://dbpedia.org/sparql',
  1,
  52]]

In [239]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 50000

In [240]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 53

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(53,
 'Cherche les occupation (dbo)',
 'Cherche les occupation (dbo)\r\n50000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?person ?occupation \r\n WHERE \r\n   {\r\n     {\r\n       SELECT DISTINCT ?person ?occupation \r\nWHERE \r\n             { ?person  dbo:occupation ?occupation\r\n                {?person  a                     yago:Intellectual109621545}\r\n             } \r\n       ORDER BY ?person \r\n     }\r\n   } \r\nOFFSET 40000 LIMIT 10000',
 None,
 '2021-07-05 00:11:16')

In [241]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [242]:
len(qr['results']['bindings'])

10000

In [243]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [244]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Michael_Lulume_Bayigga',
  'http://dbpedia.org/resource/Physician'],
 ['http://dbpedia.org/resource/Michael_Luo',
  'http://dbpedia.org/resource/Journalist'],
 ['http://dbpedia.org/resource/Michael_Luo',
  'http://dbpedia.org/resource/Michael_Luo__PersonFunction__1'],
 ['http://dbpedia.org/resource/Michael_Lynne',
  'http://dbpedia.org/resource/Michael_Lynne__PersonFunction__1'],
 ['http://dbpedia.org/resource/Michael_Lynton',
  'http://dbpedia.org/resource/Snap_Inc.']]

In [245]:
rl = [[l[0], 'http://dbpedia.org/ontology/occupation', l[1], 'https://dbpedia.org/sparql', 1, 53 ] for l in r]

In [246]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Michael_Lulume_Bayigga',
  'http://dbpedia.org/ontology/occupation',
  'http://dbpedia.org/resource/Physician',
  'https://dbpedia.org/sparql',
  1,
  53],
 ['http://dbpedia.org/resource/Michael_Luo',
  'http://dbpedia.org/ontology/occupation',
  'http://dbpedia.org/resource/Journalist',
  'https://dbpedia.org/sparql',
  1,
  53],
 ['http://dbpedia.org/resource/Michael_Luo',
  'http://dbpedia.org/ontology/occupation',
  'http://dbpedia.org/resource/Michael_Luo__PersonFunction__1',
  'https://dbpedia.org/sparql',
  1,
  53]]

In [247]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 60000

In [248]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 54

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(54,
 'Cherche les occupation (dbo)',
 'Cherche les occupation (dbo)\r\n60000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?person ?occupation \r\n WHERE \r\n   {\r\n     {\r\n       SELECT DISTINCT ?person ?occupation \r\nWHERE \r\n             { ?person  dbo:occupation ?occupation\r\n                {?person  a                     yago:Intellectual109621545}\r\n             } \r\n       ORDER BY ?person \r\n     }\r\n   } \r\nOFFSET 50000 LIMIT 10000',
 None,
 '2021-07-05 00:11:16')

In [249]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [250]:
len(qr['results']['bindings'])

10000

In [251]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [252]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Rosemary_Rogers',
  'http://dbpedia.org/resource/Novelist'],
 ['http://dbpedia.org/resource/Rosemary_Sexton',
  'http://dbpedia.org/resource/Author'],
 ['http://dbpedia.org/resource/Rosemary_Sexton',
  'http://dbpedia.org/resource/Columnist'],
 ['http://dbpedia.org/resource/Rosemary_Sexton',
  'http://dbpedia.org/resource/Rosemary_Sexton__PersonFunction__1'],
 ['http://dbpedia.org/resource/Rosemary_Woolf',
  'http://dbpedia.org/resource/Scholar']]

In [253]:
rl = [[l[0], 'http://dbpedia.org/ontology/occupation', l[1], 'https://dbpedia.org/sparql', 1, 54 ] for l in r]

In [254]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Rosemary_Rogers',
  'http://dbpedia.org/ontology/occupation',
  'http://dbpedia.org/resource/Novelist',
  'https://dbpedia.org/sparql',
  1,
  54],
 ['http://dbpedia.org/resource/Rosemary_Sexton',
  'http://dbpedia.org/ontology/occupation',
  'http://dbpedia.org/resource/Author',
  'https://dbpedia.org/sparql',
  1,
  54],
 ['http://dbpedia.org/resource/Rosemary_Sexton',
  'http://dbpedia.org/ontology/occupation',
  'http://dbpedia.org/resource/Columnist',
  'https://dbpedia.org/sparql',
  1,
  54]]

In [255]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 70000

In [256]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 55

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(55,
 'Cherche les occupation (dbo)',
 'Cherche les occupation (dbo)\r\n70000 max',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?person ?occupation \r\n WHERE \r\n   {\r\n     {\r\n       SELECT DISTINCT ?person ?occupation \r\nWHERE \r\n             { ?person  dbo:occupation ?occupation\r\n                {?person  a                     yago:Intellectual109621545}\r\n             } \r\n       ORDER BY ?person \r\n     }\r\n   } \r\nOFFSET 60000 LIMIT 10000',
 None,
 '2021-07-05 00:11:16')

In [257]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [258]:
len(qr['results']['bindings'])

1051

In [259]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [260]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

1051


[['http://dbpedia.org/resource/Willis_Adcock',
  'http://dbpedia.org/resource/Willis_Adcock__PersonFunction__1'],
 ['http://dbpedia.org/resource/Willis_Adcock',
  'http://dbpedia.org/resource/Electrical_engineering'],
 ['http://dbpedia.org/resource/Willis_Barnstone',
  'http://dbpedia.org/resource/Bible'],
 ['http://dbpedia.org/resource/Willis_Barnstone',
  'http://dbpedia.org/resource/Willis_Barnstone__PersonFunction__1'],
 ['http://dbpedia.org/resource/Willis_Barnstone',
  'http://dbpedia.org/resource/Professor']]

In [261]:
rl = [[l[0], 'http://dbpedia.org/ontology/occupation', l[1], 'https://dbpedia.org/sparql', 1, 55 ] for l in r]

In [262]:
print(len(rl))
rl[:3]

1051


[['http://dbpedia.org/resource/Willis_Adcock',
  'http://dbpedia.org/ontology/occupation',
  'http://dbpedia.org/resource/Willis_Adcock__PersonFunction__1',
  'https://dbpedia.org/sparql',
  1,
  55],
 ['http://dbpedia.org/resource/Willis_Adcock',
  'http://dbpedia.org/ontology/occupation',
  'http://dbpedia.org/resource/Electrical_engineering',
  'https://dbpedia.org/sparql',
  1,
  55],
 ['http://dbpedia.org/resource/Willis_Barnstone',
  'http://dbpedia.org/ontology/occupation',
  'http://dbpedia.org/resource/Bible',
  'https://dbpedia.org/sparql',
  1,
  55]]

In [263]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

### Sauvegarde DB

Copie4 = entités person enregistrées / requêtes birthyear, birthdate ajoutées / propriétés birthyear (dbo) birthdate (dbo) et birthdate (dbp) ajoutées / occupation (dbo) ajoutée

In [7]:
### Fonction qui récupère et met en forme le temps au moment de l'exécution

# définition
def timestamp_formatted_for_file_name():
    is_now = time.strftime('%Y%m%d_%H%M%S')
    return is_now

# exécution
timestamp_formatted_for_file_name()

'20210705_123324'

In [8]:
### Définir les adresses des fichiers, l'existant et celui à créer
original_db = 'data/intellectuels.db'

##  Noter que la différence de suffixe, en soi totalement arbitraire, 
#  dépend du fait que dans la configuration de .gitignore, .sqlite est exclu du verisonnement GIT
#  contrairement à .db qui est versionné
timestamped_db_copy = 'data/intellectuels_copie4_' + timestamp_formatted_for_file_name() + '.sqlite'

In [9]:
### Créer une copie de sauvegarde avec timestamp du fichier (versionnement manuel)
# ATTENTION : la base de données doit exister à l'endroit indiqué !
## Cette requête n'est utile que si des modifications en écriture vont être apportées à la base de données,
# afin de préserver le dernier état avant modification

## Documentation:
# https://docs.python.org/3/library/shutil.htmlcopied_db = copyfile(original_db, timestamped_db_copy)

copied_db = copyfile(original_db, timestamped_db_copy)
copied_db

'data/intellectuels_copie4_20210705_123337.sqlite'

### birthplace (dbo)

#### 10000

In [10]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 57

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(57,
 'Lieux de naissance (dbo)',
 '10000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT ?person ?birthPlace\r\nWHERE\r\n  { ?person  dbo:birthPlace  ?birthPlace\r\n      { ?person  a                     yago:Intellectual109621545}\r\n  }\r\nORDER BY ?person\r\n',
 None,
 '2021-07-05 12:34:26')

In [11]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [12]:
len(qr['results']['bindings'])

10000

In [13]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [14]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/5th_Dalai_Lama',
  'http://dbpedia.org/resource/Ü-Tsang'],
 ['http://dbpedia.org/resource/5th_Dalai_Lama',
  'http://dbpedia.org/resource/Tibet'],
 ['http://dbpedia.org/resource/5th_Dalai_Lama',
  'http://dbpedia.org/resource/Taktsé_Castle'],
 ['http://dbpedia.org/resource/9th_Wonder',
  'http://dbpedia.org/resource/Winston-Salem,_North_Carolina'],
 ["http://dbpedia.org/resource/A'Lelia_Bundles",
  'http://dbpedia.org/resource/Chicago,_Illinois']]

In [15]:
rl = [[l[0], 'http://dbpedia.org/ontology/birthPlace', l[1], 'https://dbpedia.org/sparql', 1, 57 ] for l in r]

In [16]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/5th_Dalai_Lama',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Ü-Tsang',
  'https://dbpedia.org/sparql',
  1,
  57],
 ['http://dbpedia.org/resource/5th_Dalai_Lama',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Tibet',
  'https://dbpedia.org/sparql',
  1,
  57],
 ['http://dbpedia.org/resource/5th_Dalai_Lama',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Taktsé_Castle',
  'https://dbpedia.org/sparql',
  1,
  57]]

In [17]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 20000

In [18]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 58

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(58,
 'Lieux de naissance (dbo)',
 '20000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT ?person ?birthPlace\r\nWHERE\r\n  { ?person  dbo:birthPlace  ?birthPlace\r\n      { ?person  a                     yago:Intellectual109621545}\r\n  }\r\nORDER BY ?person\r\nOFFSET 10000 LIMIT 10000',
 None,
 '2021-07-05 12:35:38')

In [19]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [20]:
len(qr['results']['bindings'])

10000

In [21]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [22]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Archie_Roy',
  'http://dbpedia.org/resource/Glasgow'],
 ['http://dbpedia.org/resource/Archie_Scott_Gobber',
  'http://dbpedia.org/resource/Warrensburg,_Missouri'],
 ['http://dbpedia.org/resource/Archie_Shepp',
  'http://dbpedia.org/resource/Fort_Lauderdale'],
 ['http://dbpedia.org/resource/Archimedes',
  'http://dbpedia.org/resource/Magna_Graecia'],
 ['http://dbpedia.org/resource/Archimedes',
  'http://dbpedia.org/resource/Syracuse,_Sicily']]

In [23]:
rl = [[l[0], 'http://dbpedia.org/ontology/birthPlace', l[1], 'https://dbpedia.org/sparql', 1, 58 ] for l in r]

In [24]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Archie_Roy',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Glasgow',
  'https://dbpedia.org/sparql',
  1,
  58],
 ['http://dbpedia.org/resource/Archie_Scott_Gobber',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Warrensburg,_Missouri',
  'https://dbpedia.org/sparql',
  1,
  58],
 ['http://dbpedia.org/resource/Archie_Shepp',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Fort_Lauderdale',
  'https://dbpedia.org/sparql',
  1,
  58]]

In [25]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 30000

In [26]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 59

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(59,
 'Lieux de naissance (dbo)',
 '30000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT ?person ?birthPlace\r\nWHERE\r\n  { ?person  dbo:birthPlace  ?birthPlace\r\n      { ?person  a                     yago:Intellectual109621545}\r\n  }\r\nORDER BY ?person\r\nOFFSET 20000 LIMIT 10000',
 None,
 '2021-07-05 12:35:38')

In [27]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [28]:
len(qr['results']['bindings'])

10000

In [29]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [30]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Charles-Eugène_Delaunay',
  'http://dbpedia.org/resource/France'],
 ['http://dbpedia.org/resource/Charles-Eusèbe_Dionne',
  'http://dbpedia.org/resource/Quebec'],
 ['http://dbpedia.org/resource/Charles-Eusèbe_Dionne',
  'http://dbpedia.org/resource/Saint-Denis-de-la-Bouteillerie,_Quebec'],
 ['http://dbpedia.org/resource/Charles-François_Dupuis',
  'http://dbpedia.org/resource/Trie-Château'],
 ['http://dbpedia.org/resource/Charles-Honoré_Laverdière',
  'http://dbpedia.org/resource/Lower_Canada']]

In [31]:
rl = [[l[0], 'http://dbpedia.org/ontology/birthPlace', l[1], 'https://dbpedia.org/sparql', 1, 59 ] for l in r]

In [32]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Charles-Eugène_Delaunay',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/France',
  'https://dbpedia.org/sparql',
  1,
  59],
 ['http://dbpedia.org/resource/Charles-Eusèbe_Dionne',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Quebec',
  'https://dbpedia.org/sparql',
  1,
  59],
 ['http://dbpedia.org/resource/Charles-Eusèbe_Dionne',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Saint-Denis-de-la-Bouteillerie,_Quebec',
  'https://dbpedia.org/sparql',
  1,
  59]]

In [33]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 40000

In [34]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 60

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(60,
 'Lieux de naissance (dbo)',
 '40000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT ?person ?birthPlace\r\nWHERE\r\n  { ?person  dbo:birthPlace  ?birthPlace\r\n      { ?person  a                     yago:Intellectual109621545}\r\n  }\r\nORDER BY ?person\r\nOFFSET 30000 LIMIT 10000',
 None,
 '2021-07-05 12:35:38')

In [35]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [36]:
len(qr['results']['bindings'])

10000

In [37]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [38]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Denis_Blundell',
  'http://dbpedia.org/resource/New_Zealand_cricket_team'],
 ['http://dbpedia.org/resource/Denis_Bowen',
  'http://dbpedia.org/resource/Kimberley,_Northern_Cape'],
 ['http://dbpedia.org/resource/Denis_Bowen',
  'http://dbpedia.org/resource/South_Africa'],
 ['http://dbpedia.org/resource/Denis_Chang',
  'http://dbpedia.org/resource/Malaysia'],
 ['http://dbpedia.org/resource/Denis_Chang',
  'http://dbpedia.org/resource/Sabah']]

In [39]:
rl = [[l[0], 'http://dbpedia.org/ontology/birthPlace', l[1], 'https://dbpedia.org/sparql', 1, 60 ] for l in r]

In [40]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Denis_Blundell',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/New_Zealand_cricket_team',
  'https://dbpedia.org/sparql',
  1,
  60],
 ['http://dbpedia.org/resource/Denis_Bowen',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Kimberley,_Northern_Cape',
  'https://dbpedia.org/sparql',
  1,
  60],
 ['http://dbpedia.org/resource/Denis_Bowen',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/South_Africa',
  'https://dbpedia.org/sparql',
  1,
  60]]

In [41]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 50000

In [42]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 61

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(61,
 'Lieux de naissance (dbo)',
 '50000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?person ?birthPlace\r\n WHERE \r\n   {\r\n     {\r\n       SELECT DISTINCT ?person ?birthPlace\r\nWHERE \r\n             { ?person  dbo:birthPlace ?birthPlace\r\n                {?person  a                     yago:Intellectual109621545}\r\n             } \r\n       ORDER BY ?person \r\n     }\r\n   } \r\nOFFSET 40000 LIMIT 10000',
 None,
 '2021-07-05 12:35:38')

In [43]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [44]:
len(qr['results']['bindings'])

10000

In [45]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [46]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Francis_E._Walter',
  'http://dbpedia.org/resource/Easton,_Pennsylvania'],
 ['http://dbpedia.org/resource/Francis_Edward_Gilman',
  'http://dbpedia.org/resource/Canada_East'],
 ['http://dbpedia.org/resource/Francis_Edward_Gilman',
  'http://dbpedia.org/resource/Danville,_Quebec'],
 ['http://dbpedia.org/resource/Francis_Edward_Hyland',
  'http://dbpedia.org/resource/Pennsylvania'],
 ['http://dbpedia.org/resource/Francis_Edward_Hyland',
  'http://dbpedia.org/resource/Philadelphia,_Pennsylvania']]

In [47]:
rl = [[l[0], 'http://dbpedia.org/ontology/birthPlace', l[1], 'https://dbpedia.org/sparql', 1, 61 ] for l in r]

In [48]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Francis_E._Walter',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Easton,_Pennsylvania',
  'https://dbpedia.org/sparql',
  1,
  61],
 ['http://dbpedia.org/resource/Francis_Edward_Gilman',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Canada_East',
  'https://dbpedia.org/sparql',
  1,
  61],
 ['http://dbpedia.org/resource/Francis_Edward_Gilman',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Danville,_Quebec',
  'https://dbpedia.org/sparql',
  1,
  61]]

In [49]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 60000

In [50]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 62

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(62,
 'Lieux de naissance (dbo)',
 '60000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?person ?birthPlace\r\n WHERE \r\n   {\r\n     {\r\n       SELECT DISTINCT ?person ?birthPlace\r\nWHERE \r\n             { ?person  dbo:birthPlace ?birthPlace\r\n                {?person  a                     yago:Intellectual109621545}\r\n             } \r\n       ORDER BY ?person \r\n     }\r\n   } \r\nOFFSET 50000 LIMIT 10000',
 None,
 '2021-07-05 12:35:38')

In [51]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [52]:
len(qr['results']['bindings'])

10000

In [53]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [54]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Harriet_Cohen',
  'http://dbpedia.org/resource/London'],
 ['http://dbpedia.org/resource/Harriet_Doerr',
  'http://dbpedia.org/resource/Pasadena,_California'],
 ['http://dbpedia.org/resource/Harriet_Elizabeth_Byrd',
  'http://dbpedia.org/resource/Cheyenne,_Wyoming'],
 ['http://dbpedia.org/resource/Harriet_G._Walker',
  'http://dbpedia.org/resource/United_States'],
 ['http://dbpedia.org/resource/Harriet_G._Walker',
  'http://dbpedia.org/resource/Ohio']]

In [55]:
rl = [[l[0], 'http://dbpedia.org/ontology/birthPlace', l[1], 'https://dbpedia.org/sparql', 1, 62 ] for l in r]

In [56]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Harriet_Cohen',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/London',
  'https://dbpedia.org/sparql',
  1,
  62],
 ['http://dbpedia.org/resource/Harriet_Doerr',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Pasadena,_California',
  'https://dbpedia.org/sparql',
  1,
  62],
 ['http://dbpedia.org/resource/Harriet_Elizabeth_Byrd',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Cheyenne,_Wyoming',
  'https://dbpedia.org/sparql',
  1,
  62]]

In [57]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 70000

In [58]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 63

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(63,
 'Lieux de naissance (dbo)',
 '70000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?person ?birthPlace\r\n WHERE \r\n   {\r\n     {\r\n       SELECT DISTINCT ?person ?birthPlace\r\nWHERE \r\n             { ?person  dbo:birthPlace ?birthPlace\r\n                {?person  a                     yago:Intellectual109621545}\r\n             } \r\n       ORDER BY ?person \r\n     }\r\n   } \r\nOFFSET 60000 LIMIT 10000',
 None,
 '2021-07-05 12:35:38')

In [59]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [60]:
len(qr['results']['bindings'])

10000

In [61]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [62]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/James_Williams_(cricketer)',
  'http://dbpedia.org/resource/Neath'],
 ['http://dbpedia.org/resource/James_Wilson_(Colorado_politician)',
  'http://dbpedia.org/resource/Kiowa,_Kansas'],
 ['http://dbpedia.org/resource/James_Wilson_(Secretary_of_Agriculture)',
  'http://dbpedia.org/resource/United_Kingdom_of_Great_Britain_and_Ireland'],
 ['http://dbpedia.org/resource/James_Wilson_(Secretary_of_Agriculture)',
  'http://dbpedia.org/resource/Scotland'],
 ['http://dbpedia.org/resource/James_Wilson_(Secretary_of_Agriculture)',
  'http://dbpedia.org/resource/Ayrshire']]

In [63]:
rl = [[l[0], 'http://dbpedia.org/ontology/birthPlace', l[1], 'https://dbpedia.org/sparql', 1, 63 ] for l in r]

In [64]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/James_Williams_(cricketer)',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Neath',
  'https://dbpedia.org/sparql',
  1,
  63],
 ['http://dbpedia.org/resource/James_Wilson_(Colorado_politician)',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Kiowa,_Kansas',
  'https://dbpedia.org/sparql',
  1,
  63],
 ['http://dbpedia.org/resource/James_Wilson_(Secretary_of_Agriculture)',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/United_Kingdom_of_Great_Britain_and_Ireland',
  'https://dbpedia.org/sparql',
  1,
  63]]

In [65]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 80000

In [66]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 64

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(64,
 'Lieux de naissance (dbo)',
 '80000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?person ?birthPlace\r\n WHERE \r\n   {\r\n     {\r\n       SELECT DISTINCT ?person ?birthPlace\r\nWHERE \r\n             { ?person  dbo:birthPlace ?birthPlace\r\n                {?person  a                     yago:Intellectual109621545}\r\n             } \r\n       ORDER BY ?person \r\n     }\r\n   } \r\nOFFSET 70000 LIMIT 10000',
 None,
 '2021-07-05 12:35:38')

In [67]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [68]:
len(qr['results']['bindings'])

10000

In [69]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [70]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/John_Yen',
  'http://dbpedia.org/resource/Hsinchu'],
 ['http://dbpedia.org/resource/John_Yen',
  'http://dbpedia.org/resource/Taiwan'],
 ['http://dbpedia.org/resource/John_Yoo',
  'http://dbpedia.org/resource/South_Korea'],
 ['http://dbpedia.org/resource/John_Yoo', 'http://dbpedia.org/resource/Seoul'],
 ['http://dbpedia.org/resource/John_Yorke_(producer)',
  'http://dbpedia.org/resource/London']]

In [71]:
rl = [[l[0], 'http://dbpedia.org/ontology/birthPlace', l[1], 'https://dbpedia.org/sparql', 1, 64 ] for l in r]

In [72]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/John_Yen',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Hsinchu',
  'https://dbpedia.org/sparql',
  1,
  64],
 ['http://dbpedia.org/resource/John_Yen',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Taiwan',
  'https://dbpedia.org/sparql',
  1,
  64],
 ['http://dbpedia.org/resource/John_Yoo',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/South_Korea',
  'https://dbpedia.org/sparql',
  1,
  64]]

In [73]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 90000

In [74]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 65

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(65,
 'Lieux de naissance (dbo)',
 '90000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?person ?birthPlace\r\n WHERE \r\n   {\r\n     {\r\n       SELECT DISTINCT ?person ?birthPlace\r\nWHERE \r\n             { ?person  dbo:birthPlace ?birthPlace\r\n                {?person  a                     yago:Intellectual109621545}\r\n             } \r\n       ORDER BY ?person \r\n     }\r\n   } \r\nOFFSET 80000 LIMIT 10000',
 None,
 '2021-07-05 12:35:38')

In [75]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [76]:
len(qr['results']['bindings'])

10000

In [77]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [78]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Lemanu_Peleti_Mauga',
  "http://dbpedia.org/resource/Nu'uuli,_American_Samoa"],
 ['http://dbpedia.org/resource/Lemanu_Peleti_Mauga',
  'http://dbpedia.org/resource/American_Samoa'],
 ['http://dbpedia.org/resource/Lemar_Marshall',
  'http://dbpedia.org/resource/Cincinnati,_Ohio'],
 ['http://dbpedia.org/resource/Lembit_Öpik',
  'http://dbpedia.org/resource/Northern_Ireland'],
 ['http://dbpedia.org/resource/Lembit_Öpik',
  'http://dbpedia.org/resource/Bangor,_County_Down']]

In [79]:
rl = [[l[0], 'http://dbpedia.org/ontology/birthPlace', l[1], 'https://dbpedia.org/sparql', 1, 65 ] for l in r]

In [80]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Lemanu_Peleti_Mauga',
  'http://dbpedia.org/ontology/birthPlace',
  "http://dbpedia.org/resource/Nu'uuli,_American_Samoa",
  'https://dbpedia.org/sparql',
  1,
  65],
 ['http://dbpedia.org/resource/Lemanu_Peleti_Mauga',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/American_Samoa',
  'https://dbpedia.org/sparql',
  1,
  65],
 ['http://dbpedia.org/resource/Lemar_Marshall',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Cincinnati,_Ohio',
  'https://dbpedia.org/sparql',
  1,
  65]]

In [81]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 100000

In [82]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 66

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(66,
 'Lieux de naissance (dbo)',
 '100000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?person ?birthPlace\r\n WHERE \r\n   {\r\n     {\r\n       SELECT DISTINCT ?person ?birthPlace\r\nWHERE \r\n             { ?person  dbo:birthPlace ?birthPlace\r\n                {?person  a                     yago:Intellectual109621545}\r\n             } \r\n       ORDER BY ?person \r\n     }\r\n   } \r\nOFFSET 90000 LIMIT 10000',
 None,
 '2021-07-05 12:35:38')

In [83]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [84]:
len(qr['results']['bindings'])

10000

In [85]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [86]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Mei_Ju-ao',
  'http://dbpedia.org/resource/Jiangxi'],
 ['http://dbpedia.org/resource/Mei_Ju-ao',
  'http://dbpedia.org/resource/Nanchang'],
 ['http://dbpedia.org/resource/Mei_Shaowu',
  'http://dbpedia.org/resource/Beijing'],
 ['http://dbpedia.org/resource/Meic_Stephens',
  'http://dbpedia.org/resource/Glamorgan'],
 ['http://dbpedia.org/resource/Meic_Stephens',
  'http://dbpedia.org/resource/Treforest']]

In [87]:
rl = [[l[0], 'http://dbpedia.org/ontology/birthPlace', l[1], 'https://dbpedia.org/sparql', 1, 66 ] for l in r]

In [88]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Mei_Ju-ao',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Jiangxi',
  'https://dbpedia.org/sparql',
  1,
  66],
 ['http://dbpedia.org/resource/Mei_Ju-ao',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Nanchang',
  'https://dbpedia.org/sparql',
  1,
  66],
 ['http://dbpedia.org/resource/Mei_Shaowu',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Beijing',
  'https://dbpedia.org/sparql',
  1,
  66]]

In [89]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 110000

In [90]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 67

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(67,
 'Lieux de naissance (dbo)',
 '110000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?person ?birthPlace\r\n WHERE \r\n   {\r\n     {\r\n       SELECT DISTINCT ?person ?birthPlace\r\nWHERE \r\n             { ?person  dbo:birthPlace ?birthPlace\r\n                {?person  a                     yago:Intellectual109621545}\r\n             } \r\n       ORDER BY ?person \r\n     }\r\n   } \r\nOFFSET 100000 LIMIT 10000',
 None,
 '2021-07-05 12:35:38')

In [91]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [92]:
len(qr['results']['bindings'])

10000

In [93]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [94]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Otto_Neurath',
  'http://dbpedia.org/resource/Austria'],
 ['http://dbpedia.org/resource/Otto_Neurath',
  'http://dbpedia.org/resource/Austro-Hungarian_Empire'],
 ['http://dbpedia.org/resource/Otto_Neurath',
  'http://dbpedia.org/resource/Vienna'],
 ['http://dbpedia.org/resource/Otto_Newman',
  'http://dbpedia.org/resource/Vienna'],
 ['http://dbpedia.org/resource/Otto_Piene',
  'http://dbpedia.org/resource/Province_of_Westphalia']]

In [95]:
rl = [[l[0], 'http://dbpedia.org/ontology/birthPlace', l[1], 'https://dbpedia.org/sparql', 1, 67 ] for l in r]

In [96]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Otto_Neurath',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Austria',
  'https://dbpedia.org/sparql',
  1,
  67],
 ['http://dbpedia.org/resource/Otto_Neurath',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Austro-Hungarian_Empire',
  'https://dbpedia.org/sparql',
  1,
  67],
 ['http://dbpedia.org/resource/Otto_Neurath',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Vienna',
  'https://dbpedia.org/sparql',
  1,
  67]]

In [97]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 120000

In [98]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 68

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(68,
 'Lieux de naissance (dbo)',
 '120000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?person ?birthPlace\r\n WHERE \r\n   {\r\n     {\r\n       SELECT DISTINCT ?person ?birthPlace\r\nWHERE \r\n             { ?person  dbo:birthPlace ?birthPlace\r\n                {?person  a                     yago:Intellectual109621545}\r\n             } \r\n       ORDER BY ?person \r\n     }\r\n   } \r\nOFFSET 110000 LIMIT 10000',
 None,
 '2021-07-05 12:35:38')

In [99]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [100]:
len(qr['results']['bindings'])

10000

In [101]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [102]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Richard_P._Gabriel',
  'http://dbpedia.org/resource/Merrimac,_Massachusetts'],
 ['http://dbpedia.org/resource/Richard_P._McCormick',
  'http://dbpedia.org/resource/Ridgewood,_Queens'],
 ['http://dbpedia.org/resource/Richard_P._Mills',
  'http://dbpedia.org/resource/Paris,_Illinois'],
 ['http://dbpedia.org/resource/Richard_P._Rubinstein',
  'http://dbpedia.org/resource/Brooklyn'],
 ['http://dbpedia.org/resource/Richard_P._Stanley',
  'http://dbpedia.org/resource/New_York_City']]

In [103]:
rl = [[l[0], 'http://dbpedia.org/ontology/birthPlace', l[1], 'https://dbpedia.org/sparql', 1, 68 ] for l in r]

In [104]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Richard_P._Gabriel',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Merrimac,_Massachusetts',
  'https://dbpedia.org/sparql',
  1,
  68],
 ['http://dbpedia.org/resource/Richard_P._McCormick',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Ridgewood,_Queens',
  'https://dbpedia.org/sparql',
  1,
  68],
 ['http://dbpedia.org/resource/Richard_P._Mills',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Paris,_Illinois',
  'https://dbpedia.org/sparql',
  1,
  68]]

In [105]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 130000

In [106]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 69

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(69,
 'Lieux de naissance (dbo)',
 '130000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?person ?birthPlace\r\n WHERE \r\n   {\r\n     {\r\n       SELECT DISTINCT ?person ?birthPlace\r\nWHERE \r\n             { ?person  dbo:birthPlace ?birthPlace\r\n                {?person  a                     yago:Intellectual109621545}\r\n             } \r\n       ORDER BY ?person \r\n     }\r\n   } \r\nOFFSET 120000 LIMIT 10000',
 None,
 '2021-07-05 12:35:38')

In [107]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [108]:
len(qr['results']['bindings'])

10000

In [109]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [110]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Shoshana_Kamin',
  'http://dbpedia.org/resource/Moscow'],
 ['http://dbpedia.org/resource/Shoshanna_Lonstein_Gruss',
  'http://dbpedia.org/resource/United_States'],
 ['http://dbpedia.org/resource/Shoshanna_Lonstein_Gruss',
  'http://dbpedia.org/resource/New_York_(state)'],
 ['http://dbpedia.org/resource/Shoshanna_Lonstein_Gruss',
  'http://dbpedia.org/resource/Manhattan'],
 ['http://dbpedia.org/resource/Shota_Ishimine',
  'http://dbpedia.org/resource/Okinawa_Prefecture']]

In [111]:
rl = [[l[0], 'http://dbpedia.org/ontology/birthPlace', l[1], 'https://dbpedia.org/sparql', 1, 69 ] for l in r]

In [112]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Shoshana_Kamin',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Moscow',
  'https://dbpedia.org/sparql',
  1,
  69],
 ['http://dbpedia.org/resource/Shoshanna_Lonstein_Gruss',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/United_States',
  'https://dbpedia.org/sparql',
  1,
  69],
 ['http://dbpedia.org/resource/Shoshanna_Lonstein_Gruss',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/New_York_(state)',
  'https://dbpedia.org/sparql',
  1,
  69]]

In [113]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 140000

In [114]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 70

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(70,
 'Lieux de naissance (dbo)',
 '140000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?person ?birthPlace\r\n WHERE \r\n   {\r\n     {\r\n       SELECT DISTINCT ?person ?birthPlace\r\nWHERE \r\n             { ?person  dbo:birthPlace ?birthPlace\r\n                {?person  a                     yago:Intellectual109621545}\r\n             } \r\n       ORDER BY ?person \r\n     }\r\n   } \r\nOFFSET 130000 LIMIT 10000',
 None,
 '2021-07-05 12:35:38')

In [115]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [116]:
len(qr['results']['bindings'])

10000

In [117]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [118]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Toyin_Falola',
  'http://dbpedia.org/resource/Ibadan'],
 ['http://dbpedia.org/resource/Trace_Adkins',
  'http://dbpedia.org/resource/Sarepta,_Louisiana'],
 ['http://dbpedia.org/resource/Tracee_Carrasco',
  'http://dbpedia.org/resource/United_States'],
 ['http://dbpedia.org/resource/Tracee_Carrasco',
  'http://dbpedia.org/resource/Honolulu'],
 ['http://dbpedia.org/resource/Tracee_Carrasco',
  'http://dbpedia.org/resource/Hawaii']]

In [119]:
rl = [[l[0], 'http://dbpedia.org/ontology/birthPlace', l[1], 'https://dbpedia.org/sparql', 1, 70 ] for l in r]

In [120]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Toyin_Falola',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Ibadan',
  'https://dbpedia.org/sparql',
  1,
  70],
 ['http://dbpedia.org/resource/Trace_Adkins',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Sarepta,_Louisiana',
  'https://dbpedia.org/sparql',
  1,
  70],
 ['http://dbpedia.org/resource/Tracee_Carrasco',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/United_States',
  'https://dbpedia.org/sparql',
  1,
  70]]

In [121]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 150000

In [122]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 71

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(71,
 'Lieux de naissance (dbo)',
 '150000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?person ?birthPlace\r\n WHERE \r\n   {\r\n     {\r\n       SELECT DISTINCT ?person ?birthPlace\r\nWHERE \r\n             { ?person  dbo:birthPlace ?birthPlace\r\n                {?person  a                     yago:Intellectual109621545}\r\n             } \r\n       ORDER BY ?person \r\n     }\r\n   } \r\nOFFSET 140000 LIMIT 10000',
 None,
 '2021-07-05 12:35:38')

In [123]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [124]:
len(qr['results']['bindings'])

128

In [125]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [126]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

128


[['http://dbpedia.org/resource/İdris_Güllüce',
  'http://dbpedia.org/resource/Turkey'],
 ['http://dbpedia.org/resource/İdris_Güllüce',
  'http://dbpedia.org/resource/Pasinler,_Erzurum'],
 ['http://dbpedia.org/resource/İdris_Güllüce',
  'http://dbpedia.org/resource/Erzurum_Province'],
 ['http://dbpedia.org/resource/İhsan_Doğramacı',
  'http://dbpedia.org/resource/Arbil'],
 ['http://dbpedia.org/resource/İhsan_Doğramacı',
  'http://dbpedia.org/resource/Ottoman_Empire']]

In [127]:
rl = [[l[0], 'http://dbpedia.org/ontology/birthPlace', l[1], 'https://dbpedia.org/sparql', 1, 71 ] for l in r]

In [128]:
print(len(rl))
rl[:3]

128


[['http://dbpedia.org/resource/İdris_Güllüce',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Turkey',
  'https://dbpedia.org/sparql',
  1,
  71],
 ['http://dbpedia.org/resource/İdris_Güllüce',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Pasinler,_Erzurum',
  'https://dbpedia.org/sparql',
  1,
  71],
 ['http://dbpedia.org/resource/İdris_Güllüce',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Erzurum_Province',
  'https://dbpedia.org/sparql',
  1,
  71]]

In [129]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

### Sauvegarde DB

Copie5 = entités person enregistrées / requêtes birthyear, birthdate ajoutées / propriétés birthyear (dbo) birthdate (dbo) et birthdate (dbp) ajoutées / occupation (dbo) ajoutée / birthplace (dbo) ajoutée

In [130]:
### Fonction qui récupère et met en forme le temps au moment de l'exécution

# définition
def timestamp_formatted_for_file_name():
    is_now = time.strftime('%Y%m%d_%H%M%S')
    return is_now

# exécution
timestamp_formatted_for_file_name()

'20210705_131025'

In [131]:
### Définir les adresses des fichiers, l'existant et celui à créer
original_db = 'data/intellectuels.db'

##  Noter que la différence de suffixe, en soi totalement arbitraire, 
#  dépend du fait que dans la configuration de .gitignore, .sqlite est exclu du verisonnement GIT
#  contrairement à .db qui est versionné
timestamped_db_copy = 'data/intellectuels_copie5_' + timestamp_formatted_for_file_name() + '.sqlite'

In [132]:
### Créer une copie de sauvegarde avec timestamp du fichier (versionnement manuel)
# ATTENTION : la base de données doit exister à l'endroit indiqué !
## Cette requête n'est utile que si des modifications en écriture vont être apportées à la base de données,
# afin de préserver le dernier état avant modification

## Documentation:
# https://docs.python.org/3/library/shutil.htmlcopied_db = copyfile(original_db, timestamped_db_copy)

copied_db = copyfile(original_db, timestamped_db_copy)
copied_db

'data/intellectuels_copie5_20210705_131031.sqlite'

### birthplace type 

#### 10000

In [144]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 72

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(72,
 'type des lieux de naissance',
 '10000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT ?birthPlace ?type\r\nWHERE\r\n  {   ?birthPlace  a  ?type\r\n\r\n    { ?person  dbo:birthPlace  ?birthPlace\r\n      { ?person  a                     yago:Intellectual109621545 }\r\n    }\r\nFILTER(STRSTARTS(STR(?type), "http://dbpedia.org/onto") AND ?type != dbo:Location \r\n    AND ?type != dbo:Place AND  ?type != dbo:PopulatedPlace\r\n    AND  ?type != dbo:Settlement\r\n    )\r\n\r\n# FILTER (\'Place\' NOT IN (STR(?type)))\r\n# FILTER NOT EXISTS { ?birthPlace  a  dbo:Location}\r\n# MINUS { ?birthPlace  a  dbo:Place}\r\n  }\r\nORDER BY ?birthPlace \r\n',
 None,
 '2021-07-05 13:27:38')

In [145]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [146]:
len(qr['results']['bindings'])

10000

In [147]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [148]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[["http://dbpedia.org/resource/'Asir_Region",
  'http://dbpedia.org/ontology/Region'],
 ["http://dbpedia.org/resource/'Asir_Region",
  'http://dbpedia.org/ontology/AdministrativeRegion'],
 ["http://dbpedia.org/resource/'s-Graveland",
  'http://dbpedia.org/ontology/Village'],
 ["http://dbpedia.org/resource/'s-Hertogenbosch",
  'http://dbpedia.org/ontology/City'],
 ['http://dbpedia.org/resource/16th_arrondissement_of_Paris',
  'http://dbpedia.org/ontology/Town']]

In [149]:
rl = [[l[0], 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type', l[1], 'https://dbpedia.org/sparql', 1, 72 ] for l in r]

In [150]:
print(len(rl))
rl[:3]

10000


[["http://dbpedia.org/resource/'Asir_Region",
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://dbpedia.org/ontology/Region',
  'https://dbpedia.org/sparql',
  1,
  72],
 ["http://dbpedia.org/resource/'Asir_Region",
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://dbpedia.org/ontology/AdministrativeRegion',
  'https://dbpedia.org/sparql',
  1,
  72],
 ["http://dbpedia.org/resource/'s-Graveland",
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://dbpedia.org/ontology/Village',
  'https://dbpedia.org/sparql',
  1,
  72]]

In [151]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 20000

In [17]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 73

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(73,
 'type des lieux de naissance',
 '20000 ',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT ?birthPlace ?type\r\nWHERE\r\n  {   ?birthPlace  a  ?type\r\n\r\n    { ?person  dbo:birthPlace  ?birthPlace\r\n      { ?person  a                     yago:Intellectual109621545 }\r\n    }\r\nFILTER(STRSTARTS(STR(?type), "http://dbpedia.org/onto") AND ?type != dbo:Location \r\n    AND ?type != dbo:Place AND  ?type != dbo:PopulatedPlace\r\n    AND  ?type != dbo:Settlement\r\n    )\r\n\r\n# FILTER (\'Place\' NOT IN (STR(?type)))\r\n# FILTER NOT EXISTS { ?birthPlace  a  dbo:Location}\r\n# MINUS { ?birthPlace  a  dbo:Place}\r\n  }\r\nORDER BY ?birthPlace \r\nOFFSET 10000 LIMIT 10000',
 None,
 '2021-07-05 13:27:50')

In [18]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [19]:
len(qr['results']['bindings'])

10000

In [20]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [21]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Ligonier,_Indiana',
  'http://dbpedia.org/ontology/City'],
 ['http://dbpedia.org/resource/Ligonier,_Pennsylvania',
  'http://dbpedia.org/ontology/CityDistrict'],
 ['http://dbpedia.org/resource/Liguria', 'http://dbpedia.org/ontology/Region'],
 ['http://dbpedia.org/resource/Liguria',
  'http://dbpedia.org/ontology/AdministrativeRegion'],
 ['http://dbpedia.org/resource/Lika', 'http://dbpedia.org/ontology/Region']]

In [22]:
rl = [[l[0], 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type', l[1], 'https://dbpedia.org/sparql', 1, 73 ] for l in r]

In [23]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Ligonier,_Indiana',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://dbpedia.org/ontology/City',
  'https://dbpedia.org/sparql',
  1,
  73],
 ['http://dbpedia.org/resource/Ligonier,_Pennsylvania',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://dbpedia.org/ontology/CityDistrict',
  'https://dbpedia.org/sparql',
  1,
  73],
 ['http://dbpedia.org/resource/Liguria',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://dbpedia.org/ontology/Region',
  'https://dbpedia.org/sparql',
  1,
  73]]

In [24]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 30000

In [25]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 74

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(74,
 'type des lieux de naissance',
 '30000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT ?birthPlace ?type\r\nWHERE\r\n  {   ?birthPlace  a  ?type\r\n\r\n    { ?person  dbo:birthPlace  ?birthPlace\r\n      { ?person  a                     yago:Intellectual109621545 }\r\n    }\r\nFILTER(STRSTARTS(STR(?type), "http://dbpedia.org/onto") AND ?type != dbo:Location \r\n    AND ?type != dbo:Place AND  ?type != dbo:PopulatedPlace\r\n    AND  ?type != dbo:Settlement\r\n    )\r\n\r\n# FILTER (\'Place\' NOT IN (STR(?type)))\r\n# FILTER NOT EXISTS { ?birthPlace  a  dbo:Location}\r\n# MINUS { ?birthPlace  a  dbo:Place}\r\n  }\r\nORDER BY ?birthPlace \r\nOFFSET 20000 LIMIT 10000',
 None,
 '2021-07-06 11:06:38')

In [26]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [27]:
len(qr['results']['bindings'])

421

In [28]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [29]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

421


[['http://dbpedia.org/resource/Yamaga,_Kumamoto',
  'http://dbpedia.org/ontology/City'],
 ['http://dbpedia.org/resource/Yamaguchi_prefecture',
  'http://dbpedia.org/ontology/AdministrativeRegion'],
 ['http://dbpedia.org/resource/Yamanashi,_Yamanashi',
  'http://dbpedia.org/ontology/City'],
 ['http://dbpedia.org/resource/Yamato,_Kanagawa',
  'http://dbpedia.org/ontology/City'],
 ['http://dbpedia.org/resource/Yamatokōriyama,_Nara',
  'http://dbpedia.org/ontology/City']]

In [30]:
rl = [[l[0], 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type', l[1], 'https://dbpedia.org/sparql', 1, 74 ] for l in r]

In [31]:
print(len(rl))
rl[:3]

421


[['http://dbpedia.org/resource/Yamaga,_Kumamoto',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://dbpedia.org/ontology/City',
  'https://dbpedia.org/sparql',
  1,
  74],
 ['http://dbpedia.org/resource/Yamaguchi_prefecture',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://dbpedia.org/ontology/AdministrativeRegion',
  'https://dbpedia.org/sparql',
  1,
  74],
 ['http://dbpedia.org/resource/Yamanashi,_Yamanashi',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://dbpedia.org/ontology/City',
  'https://dbpedia.org/sparql',
  1,
  74]]

In [32]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

### birthplace long 

#### 10000

In [33]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 75

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(75,
 'Longitude',
 '10000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\nPREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>\r\n\r\nSELECT  ?birthPlace MAX(?long)\r\n\r\nWHERE\r\n  {   ?birthPlace  geo:long ?long\r\n\r\n    { ?person  dbo:birthPlace  ?birthPlace\r\n         { ?person  a                      yago:Intellectual109621545}\r\n    }\r\n  }\r\nGROUP BY ?birthPlace\r\nORDER BY ?birthPlace',
 None,
 '2021-07-06 11:21:46')

In [34]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [35]:
len(qr['results']['bindings'])

10000

In [36]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [37]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[["http://dbpedia.org/resource/'Uyayna", '46.3914'],
 ["http://dbpedia.org/resource/'s-Graveland", '5.12111'],
 ["http://dbpedia.org/resource/'s-Hertogenbosch", '5.3'],
 ['http://dbpedia.org/resource/16th_arrondissement_of_Paris', '2.27649'],
 ['http://dbpedia.org/resource/24_Parganas', '88.0']]

In [38]:
rl = [[l[0], 'http://www.w3.org/2003/01/geo/wgs84_pos#long', l[1], 'https://dbpedia.org/sparql', 1, 75 ] for l in r]

In [39]:
print(len(rl))
rl[:3]

10000


[["http://dbpedia.org/resource/'Uyayna",
  'http://www.w3.org/2003/01/geo/wgs84_pos#long',
  '46.3914',
  'https://dbpedia.org/sparql',
  1,
  75],
 ["http://dbpedia.org/resource/'s-Graveland",
  'http://www.w3.org/2003/01/geo/wgs84_pos#long',
  '5.12111',
  'https://dbpedia.org/sparql',
  1,
  75],
 ["http://dbpedia.org/resource/'s-Hertogenbosch",
  'http://www.w3.org/2003/01/geo/wgs84_pos#long',
  '5.3',
  'https://dbpedia.org/sparql',
  1,
  75]]

In [40]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 20000

In [41]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 76

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(76,
 'Longitude',
 '20000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\nPREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>\r\n\r\nSELECT  ?birthPlace MAX(?long)\r\n\r\nWHERE\r\n  {   ?birthPlace  geo:long ?long\r\n\r\n    { ?person  dbo:birthPlace  ?birthPlace\r\n         { ?person  a                      yago:Intellectual109621545}\r\n    }\r\n  }\r\nGROUP BY ?birthPlace\r\nORDER BY ?birthPlace\r\nOFFSET 10000 LIMIT 10000',
 None,
 '2021-07-06 11:21:46')

In [42]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [43]:
len(qr['results']['bindings'])

10000

In [44]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [45]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Kovačica', '20.6214'],
 ['http://dbpedia.org/resource/Kovilpatti', '77.87'],
 ['http://dbpedia.org/resource/Kovno_Governorate', '23.9333'],
 ['http://dbpedia.org/resource/Kowalewko,_Kuyavian-Pomeranian_Voivodeship',
  '17.4314'],
 ['http://dbpedia.org/resource/Kowloon', '114.183']]

In [46]:
rl = [[l[0], 'http://www.w3.org/2003/01/geo/wgs84_pos#long', l[1], 'https://dbpedia.org/sparql', 1, 76 ] for l in r]

In [47]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Kovačica',
  'http://www.w3.org/2003/01/geo/wgs84_pos#long',
  '20.6214',
  'https://dbpedia.org/sparql',
  1,
  76],
 ['http://dbpedia.org/resource/Kovilpatti',
  'http://www.w3.org/2003/01/geo/wgs84_pos#long',
  '77.87',
  'https://dbpedia.org/sparql',
  1,
  76],
 ['http://dbpedia.org/resource/Kovno_Governorate',
  'http://www.w3.org/2003/01/geo/wgs84_pos#long',
  '23.9333',
  'https://dbpedia.org/sparql',
  1,
  76]]

In [48]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 30000

In [49]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 77

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(77,
 'Longitude',
 '30000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\nPREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>\r\n\r\nSELECT  ?birthPlace MAX(?long)\r\n\r\nWHERE\r\n  {   ?birthPlace  geo:long ?long\r\n\r\n    { ?person  dbo:birthPlace  ?birthPlace\r\n         { ?person  a                      yago:Intellectual109621545}\r\n    }\r\n  }\r\nGROUP BY ?birthPlace\r\nORDER BY ?birthPlace\r\nOFFSET 20000 LIMIT 10000',
 None,
 '2021-07-06 11:21:46')

In [50]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [51]:
len(qr['results']['bindings'])

1752

In [52]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [53]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

1752


[['http://dbpedia.org/resource/Victorica', '-65.45'],
 ['http://dbpedia.org/resource/Victorville,_California', '-117.291'],
 ['http://dbpedia.org/resource/Vicuña,_Chile', '-70.7167'],
 ['http://dbpedia.org/resource/Vidin', '22.8667'],
 ['http://dbpedia.org/resource/Vidin_Province', '22.6833']]

In [54]:
rl = [[l[0], 'http://www.w3.org/2003/01/geo/wgs84_pos#long', l[1], 'https://dbpedia.org/sparql', 1, 77 ] for l in r]

In [55]:
print(len(rl))
rl[:3]

1752


[['http://dbpedia.org/resource/Victorica',
  'http://www.w3.org/2003/01/geo/wgs84_pos#long',
  '-65.45',
  'https://dbpedia.org/sparql',
  1,
  77],
 ['http://dbpedia.org/resource/Victorville,_California',
  'http://www.w3.org/2003/01/geo/wgs84_pos#long',
  '-117.291',
  'https://dbpedia.org/sparql',
  1,
  77],
 ['http://dbpedia.org/resource/Vicuña,_Chile',
  'http://www.w3.org/2003/01/geo/wgs84_pos#long',
  '-70.7167',
  'https://dbpedia.org/sparql',
  1,
  77]]

In [56]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! 
cn.commit()

cn.close()

### birthplace lat 

#### 10000

In [57]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 78

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(78,
 'Latitude',
 '10000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\nPREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>\r\n\r\nSELECT  ?birthPlace MAX(?lat)\r\n\r\nWHERE\r\n  {   ?birthPlace  geo:lat ?lat\r\n\r\n    { ?person  dbo:birthPlace  ?birthPlace\r\n         { ?person  a                      yago:Intellectual109621545}\r\n    }\r\n  }\r\nGROUP BY ?birthPlace\r\nORDER BY ?birthPlace',
 None,
 None)

In [58]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [59]:
len(qr['results']['bindings'])

10000

In [60]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [61]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[["http://dbpedia.org/resource/'Uyayna", '24.9053'],
 ["http://dbpedia.org/resource/'s-Graveland", '52.2442'],
 ["http://dbpedia.org/resource/'s-Hertogenbosch", '51.6833'],
 ['http://dbpedia.org/resource/16th_arrondissement_of_Paris', '48.8636'],
 ['http://dbpedia.org/resource/24_Parganas', '22.0']]

In [62]:
rl = [[l[0], 'http://www.w3.org/2003/01/geo/wgs84_pos#lat', l[1], 'https://dbpedia.org/sparql', 1, 78 ] for l in r]

In [63]:
print(len(rl))
rl[:3]

10000


[["http://dbpedia.org/resource/'Uyayna",
  'http://www.w3.org/2003/01/geo/wgs84_pos#lat',
  '24.9053',
  'https://dbpedia.org/sparql',
  1,
  78],
 ["http://dbpedia.org/resource/'s-Graveland",
  'http://www.w3.org/2003/01/geo/wgs84_pos#lat',
  '52.2442',
  'https://dbpedia.org/sparql',
  1,
  78],
 ["http://dbpedia.org/resource/'s-Hertogenbosch",
  'http://www.w3.org/2003/01/geo/wgs84_pos#lat',
  '51.6833',
  'https://dbpedia.org/sparql',
  1,
  78]]

In [64]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 20000

In [65]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 79

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(79,
 'Latitude',
 '20000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\nPREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>\r\n\r\nSELECT  ?birthPlace MAX(?lat)\r\n\r\nWHERE\r\n  {   ?birthPlace  geo:lat ?lat\r\n\r\n    { ?person  dbo:birthPlace  ?birthPlace\r\n         { ?person  a                      yago:Intellectual109621545}\r\n    }\r\n  }\r\nGROUP BY ?birthPlace\r\nORDER BY ?birthPlace\r\nOFFSET 10000 LIMIT 10000',
 None,
 '2021-07-06 11:31:18')

In [66]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [67]:
len(qr['results']['bindings'])

10000

In [68]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [69]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

10000


[['http://dbpedia.org/resource/Kovačica', '45.1117'],
 ['http://dbpedia.org/resource/Kovilpatti', '9.17'],
 ['http://dbpedia.org/resource/Kovno_Governorate', '54.9'],
 ['http://dbpedia.org/resource/Kowalewko,_Kuyavian-Pomeranian_Voivodeship',
  '53.0719'],
 ['http://dbpedia.org/resource/Kowloon', '22.3167']]

In [70]:
rl = [[l[0], 'http://www.w3.org/2003/01/geo/wgs84_pos#lat', l[1], 'https://dbpedia.org/sparql', 1, 79 ] for l in r]

In [71]:
print(len(rl))
rl[:3]

10000


[['http://dbpedia.org/resource/Kovačica',
  'http://www.w3.org/2003/01/geo/wgs84_pos#lat',
  '45.1117',
  'https://dbpedia.org/sparql',
  1,
  79],
 ['http://dbpedia.org/resource/Kovilpatti',
  'http://www.w3.org/2003/01/geo/wgs84_pos#lat',
  '9.17',
  'https://dbpedia.org/sparql',
  1,
  79],
 ['http://dbpedia.org/resource/Kovno_Governorate',
  'http://www.w3.org/2003/01/geo/wgs84_pos#lat',
  '54.9',
  'https://dbpedia.org/sparql',
  1,
  79]]

In [72]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

#### 30000

In [73]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 80

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(80,
 'Latitude',
 '30000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\nPREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>\r\n\r\nSELECT  ?birthPlace MAX(?lat)\r\n\r\nWHERE\r\n  {   ?birthPlace  geo:lat ?lat\r\n\r\n    { ?person  dbo:birthPlace  ?birthPlace\r\n         { ?person  a                      yago:Intellectual109621545}\r\n    }\r\n  }\r\nGROUP BY ?birthPlace\r\nORDER BY ?birthPlace\r\nOFFSET 20000 LIMIT 10000',
 None,
 '2021-07-06 11:31:18')

In [74]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [75]:
len(qr['results']['bindings'])

1752

In [76]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [77]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

1752


[['http://dbpedia.org/resource/Victorica', '-36.2167'],
 ['http://dbpedia.org/resource/Victorville,_California', '34.5361'],
 ['http://dbpedia.org/resource/Vicuña,_Chile', '-30.0333'],
 ['http://dbpedia.org/resource/Vidin', '44.0'],
 ['http://dbpedia.org/resource/Vidin_Province', '43.8']]

In [78]:
rl = [[l[0], 'http://www.w3.org/2003/01/geo/wgs84_pos#lat', l[1], 'https://dbpedia.org/sparql', 1, 80 ] for l in r]

In [79]:
print(len(rl))
rl[:3]

1752


[['http://dbpedia.org/resource/Victorica',
  'http://www.w3.org/2003/01/geo/wgs84_pos#lat',
  '-36.2167',
  'https://dbpedia.org/sparql',
  1,
  80],
 ['http://dbpedia.org/resource/Victorville,_California',
  'http://www.w3.org/2003/01/geo/wgs84_pos#lat',
  '34.5361',
  'https://dbpedia.org/sparql',
  1,
  80],
 ['http://dbpedia.org/resource/Vicuña,_Chile',
  'http://www.w3.org/2003/01/geo/wgs84_pos#lat',
  '-30.0333',
  'https://dbpedia.org/sparql',
  1,
  80]]

In [80]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

### Sauvegarde DB

Copie6 = entités person enregistrées / requêtes birthyear, birthdate ajoutées / propriétés birthyear (dbo) birthdate (dbo) et birthdate (dbp) ajoutées / occupation (dbo) ajoutée / birthplace (dbo) ajoutée / birthplace type, long, lat ajoutés

In [81]:
### Fonction qui récupère et met en forme le temps au moment de l'exécution

# définition
def timestamp_formatted_for_file_name():
    is_now = time.strftime('%Y%m%d_%H%M%S')
    return is_now

# exécution
timestamp_formatted_for_file_name()

'20210706_113933'

In [82]:
### Définir les adresses des fichiers, l'existant et celui à créer
original_db = 'data/intellectuels.db'

##  Noter que la différence de suffixe, en soi totalement arbitraire, 
#  dépend du fait que dans la configuration de .gitignore, .sqlite est exclu du verisonnement GIT
#  contrairement à .db qui est versionné
timestamped_db_copy = 'data/intellectuels_copie6_' + timestamp_formatted_for_file_name() + '.sqlite'

In [83]:
### Créer une copie de sauvegarde avec timestamp du fichier (versionnement manuel)
# ATTENTION : la base de données doit exister à l'endroit indiqué !
## Cette requête n'est utile que si des modifications en écriture vont être apportées à la base de données,
# afin de préserver le dernier état avant modification

## Documentation:
# https://docs.python.org/3/library/shutil.htmlcopied_db = copyfile(original_db, timestamped_db_copy)

copied_db = copyfile(original_db, timestamped_db_copy)
copied_db

'data/intellectuels_copie6_20210706_113941.sqlite'

## Enregistrer les entitées 'place'

In [98]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 83

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(83,
 'Liste des lieux de naissance',
 '30000',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?birthPlace\r\nWHERE\r\n  { ?person  dbo:birthPlace  ?birthPlace\r\n      { ?person  a                     yago:Intellectual109621545 }\r\n  }\r\nORDER BY ?birthPlace\r\nOFFSET 20000 LIMIT 10000',
 None,
 '2021-07-06 11:52:36')

In [99]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [100]:
len(qr['results']['bindings'])

7730

In [101]:
r = [l for l in spqf.sparql_result_to_list(qr) ]

In [102]:
rl = [[l[0], 'https://dbpedia.org/ontology/Place', 'https://dbpedia.org/sparql', 83 ] for l in r]

In [103]:
rl[:3]

[['http://dbpedia.org/resource/Ramanathapuram_district',
  'https://dbpedia.org/ontology/Place',
  'https://dbpedia.org/sparql',
  83],
 ['http://dbpedia.org/resource/Ramankary',
  'https://dbpedia.org/ontology/Place',
  'https://dbpedia.org/sparql',
  83],
 ['http://dbpedia.org/resource/Ramapur',
  'https://dbpedia.org/ontology/Place',
  'https://dbpedia.org/sparql',
  83]]

In [104]:
### Stocker les nouvelles entités en base de données – ne pas oublier d'active le 'commit' !

cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO entity (uri_entity, entity_class, source, fk_query_as_source) VALUES (?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! ##  cn.commit()

cn.close()

### Sauvegarde DB

Copie7 = entités person enregistrées / requêtes birthyear, birthdate ajoutées / propriétés birthyear (dbo) birthdate (dbo) et birthdate (dbp) ajoutées / occupation (dbo) ajoutée / birthplace (dbo) ajoutée / birthplace type, long, lat ajoutés / entités place enregistrées 

In [105]:
### Fonction qui récupère et met en forme le temps au moment de l'exécution

# définition
def timestamp_formatted_for_file_name():
    is_now = time.strftime('%Y%m%d_%H%M%S')
    return is_now

# exécution
timestamp_formatted_for_file_name()

'20210706_132647'

In [106]:
### Définir les adresses des fichiers, l'existant et celui à créer
original_db = 'data/intellectuels.db'

##  Noter que la différence de suffixe, en soi totalement arbitraire, 
#  dépend du fait que dans la configuration de .gitignore, .sqlite est exclu du verisonnement GIT
#  contrairement à .db qui est versionné
timestamped_db_copy = 'data/intellectuels_copie7_' + timestamp_formatted_for_file_name() + '.sqlite'

In [107]:
### Créer une copie de sauvegarde avec timestamp du fichier (versionnement manuel)
# ATTENTION : la base de données doit exister à l'endroit indiqué !
## Cette requête n'est utile que si des modifications en écriture vont être apportées à la base de données,
# afin de préserver le dernier état avant modification

## Documentation:
# https://docs.python.org/3/library/shutil.htmlcopied_db = copyfile(original_db, timestamped_db_copy)

copied_db = copyfile(original_db, timestamped_db_copy)
copied_db

'data/intellectuels_copie7_20210706_132706.sqlite'

### academicDiscipline

In [13]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 87

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(87,
 'Academic discipline ',
 '20000 max',
 None,
 'https://dbpedia.org/sparql',
 'PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT ?person ?academicDiscipline\r\nWHERE\r\n  { ?person  dbo:academicDiscipline  ?academicDiscipline\r\n      { ?person  a                     yago:Intellectual109621545}\r\n  }\r\n ORDER BY ?person\r\n OFFSET 10000 LIMIT 10000',
 None,
 '2021-07-06 14:36:53')

In [14]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [15]:
len(qr['results']['bindings'])

2845

In [16]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [17]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

2845


[['http://dbpedia.org/resource/Richard_Kayne',
  'http://dbpedia.org/resource/Syntax'],
 ['http://dbpedia.org/resource/Richard_Kayne',
  'http://dbpedia.org/resource/Generative_grammar'],
 ['http://dbpedia.org/resource/Richard_Keynes',
  'http://dbpedia.org/resource/Physiology'],
 ['http://dbpedia.org/resource/Richard_L._Abrams',
  'http://dbpedia.org/resource/Physics'],
 ['http://dbpedia.org/resource/Richard_L._Hoffman',
  'http://dbpedia.org/resource/Millipedes']]

In [18]:
rl = [[l[0], 'http://dbpedia.org/ontology/academicDiscipline', l[1], 'https://dbpedia.org/sparql', 1, 87 ] for l in r]

In [19]:
print(len(rl))
rl[:3]

2845


[['http://dbpedia.org/resource/Richard_Kayne',
  'http://dbpedia.org/ontology/academicDiscipline',
  'http://dbpedia.org/resource/Syntax',
  'https://dbpedia.org/sparql',
  1,
  87],
 ['http://dbpedia.org/resource/Richard_Kayne',
  'http://dbpedia.org/ontology/academicDiscipline',
  'http://dbpedia.org/resource/Generative_grammar',
  'https://dbpedia.org/sparql',
  1,
  87],
 ['http://dbpedia.org/resource/Richard_Keynes',
  'http://dbpedia.org/ontology/academicDiscipline',
  'http://dbpedia.org/resource/Physiology',
  'https://dbpedia.org/sparql',
  1,
  87]]

In [20]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

### abstract

In [151]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 106

# connnexion à la base de données
original_db = 'data/intellectuels.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
rc

(106,
 'Abstract (uniquement en anglais)',
 '19 max',
 None,
 'https://dbpedia.org/sparql',
 "PREFIX  dbo:  <http://dbpedia.org/ontology/>\r\nPREFIX  dbp:  <http://dbpedia.org/property/>\r\nPREFIX  dbr:  <http://dbpedia.org/resource/>\r\nPREFIX  yago: <http://dbpedia.org/class/yago/>\r\n\r\nSELECT DISTINCT  ?person ?abstract\r\n WHERE \r\n   {\r\n     {\r\n       SELECT DISTINCT ?person ?abstract\r\nWHERE \r\n             { ?person  dbo:abstract ?abstract\r\n                {?person  a                     yago:Intellectual109621545}\r\n             } \r\n       ORDER BY ?person \r\n     }\r\nBIND (LANG(?abstract) AS ?lang)\r\nFILTER (?lang IN ('en'))\r\n   } \r\n\r\nOFFSET 180000 LIMIT 10000",
 None,
 '2021-07-06 19:58:23')

In [152]:
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [153]:
len(qr['results']['bindings'])

5583

In [154]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)]
# r

In [155]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:5]

5583


[['http://dbpedia.org/resource/William_H._Wilbur',
  "William Hale Wilbur (September 24, 1888 – December 27, 1979) was a United States Army officer and a recipient of the United States military's highest decoration—the Medal of Honor—for his actions in World War II."],
 ['http://dbpedia.org/resource/William_H._Wiley',
  "William Halsted Wiley (July 10, 1842 in New York City – May 2, 1925 in Orange, New Jersey), was an American Republican Party politician who represented New Jersey's 8th congressional district from 1903 to 1907 and from 1909 to 1911, and was also a co-founder and former president of the publishing company John Wiley & Sons."],
 ['http://dbpedia.org/resource/William_H._Woodin',
  'William Hartman Woodin (May 27, 1868 – May 3, 1934) was a U.S. industrialist. He served as the Secretary of Treasury under Franklin Roosevelt in 1933.'],
 ['http://dbpedia.org/resource/William_Hacket',
  'William Hacket, also known as Hackett (died 1591), was an English puritan and religious fa

In [156]:
rl = [[l[0], 'http://dbpedia.org/ontology/abstract', l[1], 'https://dbpedia.org/sparql', 1, 106 ] for l in r]

In [157]:
print(len(rl))
rl[:3]

5583


[['http://dbpedia.org/resource/William_H._Wilbur',
  'http://dbpedia.org/ontology/abstract',
  "William Hale Wilbur (September 24, 1888 – December 27, 1979) was a United States Army officer and a recipient of the United States military's highest decoration—the Medal of Honor—for his actions in World War II.",
  'https://dbpedia.org/sparql',
  1,
  106],
 ['http://dbpedia.org/resource/William_H._Wiley',
  'http://dbpedia.org/ontology/abstract',
  "William Halsted Wiley (July 10, 1842 in New York City – May 2, 1925 in Orange, New Jersey), was an American Republican Party politician who represented New Jersey's 8th congressional district from 1903 to 1907 and from 1909 to 1911, and was also a co-founder and former president of the publishing company John Wiley & Sons.",
  'https://dbpedia.org/sparql',
  1,
  106],
 ['http://dbpedia.org/resource/William_H._Woodin',
  'http://dbpedia.org/ontology/abstract',
  'William Hartman Woodin (May 27, 1868 – May 3, 1934) was a U.S. industrialist. He 

In [158]:
#STOCKAGE
db = 'data/intellectuels.db'
cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! 
cn.commit()

cn.close()